# Phonation onset study

## Imports

In [ ]:
from typing import List, Mapping, Any
from numpy.typing import NDArray

from tqdm.notebook import tqdm
from os.path import isfile
import itertools
from pprint import pprint

import h5py
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt, gridspec
import dolfin as dfn
import pandas as pd

from femvf import statefile as sf, meshutils
from femvf.forward import integrate
from blockarray import h5utils as bh5utils, blockvec as bv
from femvf.meshutils import process_meshlabel_to_dofs, verts_from_mesh_func

from libhopf import hopf as libhopf, setup as libsetup

from main_onsetpressure import (
    ExpParamBasic, setup_dyna_model, setup_transform, make_exp_params
)

## Function definitions

### Plotting and formatting

In [ ]:
def plot_triplots(
        fig: mpl.figure.Figure, axs: List[mpl.axes.Axes],
        coordss: List[NDArray], cellss: List[NDArray], zs: List[NDArray],
        plot_type: str='tripcolor',
        **plot_kwargs
    ):
    """
    Plot a set of `tri...` type plots with shared color limits

    The plots will use the same 'v' limits so that a shared colorbar can be
    used.

    Parameters
    ----------
    fig : mpl.Figure
        The figure instance containing plots
    axs : List[mpl.Axes]
        A list of axes for each desired tri-type plot
    coords : List[np.ndarray]
        A list of mesh coordinates for each plot
    cells : List[np.ndarray]
         A list of cell connectivity information for each plot
    zs : List[np.ndarray]
        Value for color information on each plot

    Returns
    -------
    artists: List[mpl.artist.Artist]
        A list of artists created from each `tri*` plot
    """
    zmin = np.nanmin([np.nanmin(z) for z in zs])
    zmax = np.nanmax([np.nanmax(z) for z in zs])
    if 'norm' not in plot_kwargs:
        if 'vmin' not in plot_kwargs:
            plot_kwargs['vmin'] = zmin
        if 'vmax' not in plot_kwargs:
            plot_kwargs['vmax'] = zmax

    if plot_type == 'tripcolor':
        artists = [
            ax.tripcolor(*coords.T, cells, z, **plot_kwargs)
            for ax, z, coords, cells in zip(axs, zs, coordss, cellss)
        ]
    elif plot_type == 'tricontourf':
        artists = [
            ax.tricontourf(*coords.T, cells, z, **plot_kwargs)
            for ax, z, coords, cells in zip(axs, zs, coordss, cellss)
        ]
    elif plot_type == 'tricontour':
        artists = [
            ax.tricontour(*coords.T, cells, z, **plot_kwargs)
            for ax, z, coords, cells in zip(axs, zs, coordss, cellss)
        ]
    else:
        raise ValueError(f"Unknown 'plot_type' '{plot_type}'")
    return artists


def format_axes_grid(
        axs: NDArray[mpl.axes.Axes],
        xlabel: str, ylabel: str,
        sharex=True, sharey=True
    ):
    """
    Format a grid of axes so they have a shared x/y axis and label

    Parameters
    ----------
    axs : ArrayLike[n, m]
        2D array of axes objects in a grid
    """
    for ax in axs[:, 0]:
        ax.set_ylabel(ylabel)

    for ax in axs[-1, :]:
        ax.set_xlabel(xlabel)

    for ax in axs[:-1, :].flat:
        ax.xaxis.set_tick_params(labelbottom=False)
    for ax in axs[:, 1:].flat:
        ax.yaxis.set_tick_params(labelleft=False)

    if sharex:
        for ax in axs.flat:
            ax.sharex(axs.flat[0])
    if sharey:
        for ax in axs.flat:
            ax.sharey(axs.flat[0])

def format_xaxis_top(xaxis: mpl.axis.Axis):
    """
    Format an x-axis to be on the top side of a plot

    Parameters
    ----------
    xaxis: mpl.axis.Axis
    """
    xaxis.set_label_position('top')
    xaxis.set_tick_params(
        labelbottom=False, labeltop=True,
        bottom=False, top=True
    )
    return xaxis


def set_triangulation_line(
        tri_line: mpl.lines.Line2D,
        tri: mpl.tri.Triangulation,
        coords: NDArray
    ):
    """
    Set triangulation line artist coordinates

    Parameters
    ----------
    tri_line: mpl.lines.Line2D
        The `Line2D` artist returned when plotting a triangulation object
    tri: mpl.tri.Triangulation
        The `Triangulation` object representing the triangular mesh
    coords: NDArray
        The coordinates of vertices in the triangular mesh that should be set
    """
    tri_line_x = np.insert(coords[:, 0][tri.edges], 2, np.nan, axis=1)
    tri_line_y = np.insert(coords[:, 1][tri.edges], 2, np.nan, axis=1)
    tri_line.set_xdata(tri_line_x.ravel())
    tri_line.set_ydata(tri_line_y.ravel())

# Use this function to convert mesh tractions to mesh displacements
def mesh_traction_to_displacement(transform, traction):
    """
    Return a mesh displacement from a mesh traction
    """
    param = transform.x.copy()
    param[:] = 0
    param['tmesh'] = traction
    prop = transform.apply(param)
    return prop['umesh']

### Loading experiment results

In [ ]:
DEFAULT_LOAD_DIR = 'out/sensitivity'
DEFAULT_LOAD_DIR = 'out'

## Set default experiment parameters
CLSCALE = 0.5

# EMODS = np.arange(2.5, 20, 2.5) * 1e3 * 10
EMODS = 2.0 * np.arange(1, 10, 2) * 1e3 * 10

default_exp_param = {
    'MeshName': f'M5_CB_GA3_CL{CLSCALE:.2f}',
    'LayerType': 'discrete',
    'Ecov': 6.0e4,
    'Ebod': 6.0e4,
    'ParamOption': 'Stiffness',
    'Functional': 'OnsetPressure',
    'H': 1e-3,
    'EigTarget': 'LARGEST_MAGNITUDE',
    'SepPoint': 'separation-inf',
    'BifParam': 'psub'
}
DEFAULT_PARAM = ExpParamBasic(default_exp_param)

ExperimentParamDict = Mapping[str, Any]
def load_scalar_array(
        experiment_params: ExperimentParamDict,
        dataset_name: str,
        default_param: ExperimentParamDict=DEFAULT_PARAM,
        load_dir: str=DEFAULT_LOAD_DIR
    ) -> NDArray:
    """
    Load a scalar array

    Parameters
    ----------
    experiment_params: ExperimentParamDict
        Parameters of the experiment

        These parameters are combined with `default_param` for any parameters
        that are unspecified.
    dataset_name: str
        The path of the dataset to load
    default_param: ExperimentParamDict
        Defaults for unspecified experiment parameters
    load_dir: str
        The directory to load experiment results from

    Returns
    -------
    NDArray
        The dataset array or scalar
    """
    params = default_param.substitute(experiment_params)
    fpath = f'{load_dir}/{params.to_str()}.h5'
    with h5py.File(fpath, mode='r') as f:
        return f[dataset_name][:]

def load_block_array(
        experiment_params: ExperimentParamDict,
        group_name: str,
        default_param: ExperimentParamDict=DEFAULT_PARAM,
        load_dir: str=DEFAULT_LOAD_DIR
    ) -> NDArray[bv.BlockVector]:
    """
    Load a block vector array

    NOTE: Currently this only loads 1-dimensional arrays of block vectors.
    This could be generalized to any n-dimensional array of block arrays.

    Parameters
    ----------
    experiment_params: ExperimentParamDict
        Parameters of the experiment

        These parameters are combined with `default_param` for any parameters
        that are unspecified.
    group_name: str
        The path of the group containing the block arrays
    default_param: ExperimentParamDict
        Defaults for unspecified experiment parameters
    load_dir: str
        The directory to load experiment results from

    Returns
    -------
    NDArray[bv.BlockVector]
        The array of block arrays
    """
    params = default_param.substitute(experiment_params)
    fpath = f'{load_dir}/{params.to_str()}.h5'
    with h5py.File(fpath, mode='r') as f:
        dset_name = list(f[group_name].keys())[0]
        N = f[group_name][dset_name].shape[0]
        prop = [
            bh5utils.read_block_vector_from_group(f[group_name], nvec=n)
            for n in range(N)
        ]
        return np.array(prop)

def load_eigenpair(exp_param, sort_by='abs'):
    """
    Return eigenvalues and eigenvectors from an experiment
    """

    # Load eigenpairs of the Hessian, then sort them
    eigvals = load_scalar_array(exp_param, 'eigvals')
    eigvecs = load_block_array(exp_param, 'hess_param')

    idx_sort = argsort(eigvals, sort_by=sort_by)[::-1]
    eigvals = eigvals[idx_sort]
    eigvecs = eigvecs[idx_sort]
    return eigvals, eigvecs

In [ ]:
def load_hopf_model(exp_param):
    """
    Return the Hopf model
    """
    mesh_name = exp_param['MeshName']
    mesh_path = f'mesh/{mesh_name}.h5'
    
    hopf, res, dres = setup_dyna_model(exp_param)
    return hopf

### Linear algebra functions

In [ ]:
def argsort(
        x: NDArray[complex], axis: int=-1, sort_by: str='abs'
    ) -> NDArray[int]:
    """
    Sort an array in ascending order according to a specified criteria
    """
    kwargs = {'axis': axis}
    if sort_by == 'abs':
        idx_sort = np.argsort(np.abs(x), **kwargs)
    elif sort_by == 'real':
        idx_sort = np.argsort(np.real(x), **kwargs)
    elif sort_by == 'imag':
        idx_sort = np.argsort(np.imag(x), **kwargs)
    else:
        raise ValueError(f"Unknown `sort_by` string, {sort_by}")

    return idx_sort

def form_basis(basis_vectors: List[NDArray]):
    """
    Return a matrix with columns from basis vectors
    """
    return np.stack(basis_vectors, axis=-1)

def form_projector(Z: NDArray[float], A: NDArray[float]=np.array(1.0)):
    """
    Form the projecter onto a subspace

    Parameters
    ----------
    Z : NDArray[float] (m, n)
        The basis spanning the subspace
    A : NDArray[float] (m, m)
        The matrix representing the inner product used to define orthogonality.
        Orthogonality of vectors `u` and `v` w.r.t. `A` implies
            `u.T @ A @ v == 0`

    Returns
    -------
    NDArray[float] (m, m)
    """
    # This is the A-orthogonal projector onto the subspace spanned by `Z`
    # (basis vectors in columns) is given by:
    # See ("Numerical Linear Algebra", Trefethen and Bau, 1997)
    return Z @ np.linalg.inv(np.dot(Z.T, A).dot(Z)) @ np.dot(Z.T, A)

def solve_low_rank_eig(
        eigvals: NDArray,
        eigvecs: NDArray,
        b: NDArray
    ) -> NDArray:
    """
    Solve a linear problem (Ax = b) with matrix in eigendecomposition form

    Parameters
    ----------
    eigvals: np.NDArray (n,)
        Eigenvalues of the matrix, A, as a 1D array.
    eigvecs: np.NDArray (m, n)
        Eigenvectors of the matrix, A, as a 2D array.
    b: np.NDArray (m,)
        The right-hand side of the linear problem

    Returns
    -------
    np.NDArray (n,)
        The solution in the eigenbasis
    np.NDArray (m,)
        The solution in the original basis
    """
    # NOTE: The original matrix, A, would be presented as:
    # A = Z @ LAMBDA_EIG @ Z.T

    # You can just divide `b@eigvecs` by `eigvals` because it's a diagonal
    # matrix
    # z = np.linalg.solve(np.diag(eigvals), b@eigvecs)
    z = (b@eigvecs) / eigvals
    return z, eigvecs@z

def inner_eig(x, y, eigvals, eigvecs):
    """
    Return the inner product with respect to a matrix in eigendecomposition form

    Parameters
    ----------
    x, y: np.NDArray (..., m)
        The vectors to take the inner product of
    eigvals: np.NDArray (n,)
        Eigenvalues of the matrix, A, as a 1D array.
    eigvecs: np.NDArray (m, n)
        Eigenvectors of the matrix, A, as a 2D array.

    Returns
    -------
    np.NDArray (..., n)
        The solution in the eigenbasis
    np.NDArray (..., m)
        The solution in the original basis
    """

    # The left/right vectors represent the left/right products in the
    # inner product expression: x.T@V @ A @ V.T@y
    left_vector = x[...] @ eigvecs

    # The `[..., 0]` removes the final dimension
    right_vector = (eigvecs.T @ y[..., None])[..., 0]
    return np.sum(left_vector * eigvals * right_vector, axis=-1)

## Plot configuration

In [ ]:
INCH = 1/2.54

## These are generic plot format options
RCPARAMS = {
    'font.family': 'sans-serif',
    'legend.edgecolor': 'none'
}

## These plot format options are specific to a manuscript/presentation
FIG_STYLE = 'manuscript'
# FIG_STYLE = 'presentation'

if FIG_STYLE == 'presentation':
    FIG_LX = 14.7 * INCH
    FIG_LX_WIDE = 30 * INCH
    FIG_LY = 13 * INCH
    FIG_LY_MAX = FIG_LY

    FIG_DIR = f"fig/presentation"
    FIG_EXT = 'svg'

    _RCPARAMS = {
        'font.size': 12
    }
elif FIG_STYLE == 'manuscript':

    # These are figure sizes for different journals:
    # Biomechanics and Modelling in Mechanobiology
    # FIG_LX = 8.4 * INCH
    # FIG_LX_WIDE = 17.4 * INCH
    # FIG_LY = 10 * INCH
    # FIG_LY_MAX = 23.4*INCH

    # Journal of Biomechanical Engineering
    FIG_LX = 3.25
    FIG_LX_WIDE = 6.5
    FIG_LY = 4
    FIG_LY_MAX = 6.5

    ## PLOS Computational Biology
    # FIG_LX = 5.2
    # FIG_LX_WIDE = 7.5
    # FIG_LY = 4
    # FIG_LY_MAX = 8.75
    # FONT_SIZE = 8

    FIG_DIR = f"fig/manuscript"
    FIG_EXT = 'pdf'

    _RCPARAMS = {
        'font.size': 7
    }
else:
    raise ValueError("`FIG_STYLE` must be 'manuscript' or 'presentation'.")

RCPARAMS.update(_RCPARAMS)

mpl.rcParams.update(RCPARAMS)

COLOR_CYCLE = mpl.rcParams['axes.prop_cycle'].by_key()['color']

## Prototyping

### Line searches along gradients

In [ ]:
from main_onsetpressure import (
    objective_bv, 
    setup_transform, 
    setup_dyna_model, 
    setup_reduced_functional
)

In [ ]:
param = ExpParamBasic({
    'MeshName': f'M5_CB_GA3_CL{1.0:.2f}',
    'LayerType': 'discrete',
    'Ecov': 6.0e4,
    'Ebod': 6.0e4,
    'ParamOption': 'TractionShape',
    'Functional': 'OnsetPressure',
    'H': 1e-3,
    'EigTarget': 'LARGEST_MAGNITUDE',
    'SepPoint': 'separation-inf',
    'BifParam': 'psub'
})

In [ ]:
red_func, hopf_model, xhopf, p_0, transform = setup_reduced_functional(
    param,
    lambda_tol=10.0,
    lambda_intervals=10*np.array([0, 500, 1000, 1500])
)

In [ ]:
# p = transform.x.copy()

f_0, grad_0 = objective_bv(p_0, red_func, transform)

In [ ]:
print(f_0)
grad_0.print_summary()

In [ ]:
hs = np.linspace(-200, 0, 11)
# hs = np.array([-1, 0, 1])

results = [objective_bv(p_0 + h*grad_0, red_func, transform) for h in tqdm(hs)]
fs = [res[0] for res in results]
grads = [res[1] for res in results]

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(hs, fs)
ax.set_xticks(hs)

ax.set_ylabel(r"$p_\mathrm{on}$")
ax.set_xlabel(r"$h$")

In [ ]:
p_1 = p_0 + -60* grad
f_1, grad_1 = objective_bv(p_1, red_func, transform)

In [ ]:
hs = np.linspace(-10, 0, 11)
# hs = np.array([-1, 0, 1])

results = [objective_bv(p_1 + h*grad_1, red_func, transform) for h in tqdm(hs)]
fs = [res[0] for res in results]
grads = [res[1] for res in results]

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(hs, fs)
ax.set_xticks(hs)

ax.set_ylabel(r"$p_\mathrm{on}$")
ax.set_xlabel(r"$h$")

## Main results

### Global configuration

In [ ]:
HOPF_MODEL, *_ = setup_dyna_model(DEFAULT_PARAM.substitute({'BifParam': 'psub'}))
HOPF_MODEL_Q, *_ = setup_dyna_model(DEFAULT_PARAM.substitute({'BifParam': 'qsub'}))
MESH = HOPF_MODEL.res.solid.residual.mesh()

VEC_SPACE_CG1 = HOPF_MODEL.res.solid.residual.form['coeff.state.u1'].function_space()
SCL_SPACE_CG1 = HOPF_MODEL.res.solid.residual.form['coeff.fsi.p1'].function_space()
SCL_SPACE_DG0 = HOPF_MODEL.res.solid.residual.form['coeff.prop.emod'].function_space()

VEC_CG1_DOFMAP = VEC_SPACE_CG1.dofmap()
CG1_DOFMAP = SCL_SPACE_CG1.dofmap()
DG0_DOFMAP = SCL_SPACE_DG0.dofmap()
CELL_TO_SDOF_CG1 = CG1_DOFMAP.entity_closure_dofs(MESH, 2)
CELL_TO_SDOF = DG0_DOFMAP.entity_closure_dofs(MESH, 2)

In [ ]:
MESH_NAMES = (
    [f'M5_CB_GA3_CL{clscale:.2f}' for clscale in (1.0, 0.5, 0.25, 0.125)]
    + [f'M5_CB_GA3_CL{clscale:.2f}_split' for clscale in (0.5, 0.25, 0.125)]
    + [f'M5_CB_GA3_CL{clscale:.2f}_split6' for clscale in (0.5,)]
)

# This loads all Hopf models across different meshes
sep_points = (
    4*['separation-inf'] + 3*['separation-inf'] + ['sep1']
)
HOPF_MODELS = {
    mesh_name: setup_dyna_model(
        DEFAULT_PARAM.substitute({
            'MeshName': mesh_name, 'SepPoint': sep_point
        })
    )[0]
    for mesh_name, sep_point in zip(MESH_NAMES, sep_points)
}

VDG0S = {
    mesh_name: dfn.FunctionSpace(model.res.solid.residual.mesh(), 'DG', 0)
    for mesh_name, model in HOPF_MODELS.items()
}
MEASURES = {
    mesh_name: dfn.Measure('dx', model.res.solid.residual.mesh())
    for mesh_name, model in HOPF_MODELS.items()
}
MS_DG0 = {
    mesh_name: dfn.assemble(
        dfn.TrialFunction(v)*dfn.TestFunction(v)*MEASURES[mesh_name],
        tensor=dfn.PETScMatrix()
    ).mat()
    for mesh_name, v in VDG0S.items()
}

In [ ]:
# Get a list of vertices on the medial region
residual = HOPF_MODEL.res.solid.residual

cell_func = residual.mesh_function('cell')
cell_label_to_id = residual.mesh_function_label_to_value('cell')
facet_func = residual.mesh_function('facet')
facet_label_to_id = residual.mesh_function_label_to_value('facet')
print(facet_label_to_id, cell_label_to_id)
VERTS_MED = verts_from_mesh_func(MESH, facet_func, 3)
VERTS_DIR = verts_from_mesh_func(MESH, facet_func, 4)

VERT_TO_VDOF = dfn.vertex_to_dof_map(VEC_SPACE_CG1)
VERT_TO_SDOF = dfn.vertex_to_dof_map(SCL_SPACE_CG1)

SDOF_TO_VERT = dfn.dof_to_vertex_map(SCL_SPACE_CG1)

FSPACE_CG1 = dfn.FunctionSpace(residual.mesh(), 'CG', 1)
FSPACE_DG0 = dfn.FunctionSpace(residual.mesh(), 'DG', 0)
V_DG0 = dfn.Function(FSPACE_DG0)
def project_DG0_to_CG1(f):
    V_DG0.vector()[:] = f
    return dfn.project(V_DG0, V=FSPACE_CG1).vector()[:]

### Debug: Check H5 file contents

In [ ]:
param = DEFAULT_PARAM.substitute(
    {
        'Functional': 'OnsetPressure',
        'MeshName': f'M5_CB_GA3_CL{0.5:.2f}',
        'LayerType': 'discrete',
        'Ecov': 6e4, 'Ebod': 6e4,
        'ParamOption': 'TractionShape',
        # 'ParamOption': 'Stiffness',
        'BifParam': 'psub'
    }
)

In [ ]:
fpath = f'out/sensitivity/{param.to_str()}.h5'
fpath = f'out/{param.to_str()}.h5'
with h5py.File(fpath, mode='r') as f:
    print(fpath)
    print(f.keys())
    print(f['grad_param/tmesh'])

### Plot onset pressure minimization history

In [ ]:
from main_onsetpressure import (
    setup_dyna_model, setup_transform, make_exp_params, setup_prop
)

In [ ]:
exp_param = make_exp_params('test')[0]
hopf_model = setup_dyna_model(exp_param)[0]
prop = setup_prop(exp_param, hopf_model)
transform = setup_transform(exp_param, hopf_model, prop)[0]

In [ ]:
with h5py.File('optimization_hist.h5', mode='r') as f:
    print(f.keys())
    objectives = f['objective'][:]
    print(objectives)
    params = [
        bh5utils.read_block_vector_from_group(f['parameters'], n)
        for n in range(len(objectives))
    ]

In [ ]:
ns = np.arange(len(objectives))

fig, ax = plt.subplots(1, 1)

ax.plot(ns, objs/10)
ax.set_ylim(250, 400)
ax.set_ylabel(r"Iteration")
ax.set_ylabel(r"$p_\mathrm{onset}$ $[\mathrm{Pa}]$")

In [ ]:
# Which iterates to plot
n_min = np.nanargmin(objectives)
ns = np.array([0, 1, 2, 3, 5, n_min])

props = [transform.apply(params[n]) for n in ns]
ponsets = [objectives[n] for n in ns]

sl_residual = hopf_model.res.solid.residual
mesh_cells = sl_residual.mesh().cells()
mesh_coords = sl_residual.mesh().coordinates()
vert_to_vdof = dfn.vertex_to_dof_map(sl_residual.form['coeff.prop.umesh'].function_space())
us_mesh = [prop['umesh'][vert_to_vdof].reshape(-1, 2) for prop in props]

In [ ]:
fig, axs = plt.subplots(
    1, len(ns), figsize=(3*FIG_LX_WIDE, FIG_LY),
    sharey=True
)

for ax, n, u_mesh, p_onset in zip(axs, ns, us_mesh, ponsets):
    ax.triplot(*(mesh_coords+1.0*u_mesh).T, mesh_cells, lw=1.0)
    text_summary = (
        f"$p_\\mathrm{{onset}} = {p_onset/10:.1f} [\\mathrm{{Pa}}]$\n"
        f"$n = {n}$"
    )
    ax.annotate(
        text_summary, 
        xy=(0, 1), xycoords=ax.transAxes, 
        xytext=(5, -5), textcoords='offset points', 
        va='top', ha='left'
    )

for ax in axs:
    ax.set_aspect(1)
    ax.grid()
    ax.set_xlim(0, 0.8)
    ax.set_ylim(0, 0.6)

for ax in axs:
    ax.set_xlabel(r"$x$ $[\mathrm{cm}]$")

axs[0].set_ylabel(r"$y$ $[\mathrm{cm}]$")

### Plot mesh

In [ ]:
coords = MESH.coordinates()
cells = MESH.cells()

tri = mpl.tri.Triangulation(*coords.T, cells)

In [ ]:
## Plot the mesh for making the model schematic
fig, ax = plt.subplots(1, 1, figsize=(FIG_LX, FIG_LY))

ax.triplot(tri, label='mesh')
ax.set_aspect(1)

ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")
# ax.legend()
fig.savefig(f'{FIG_DIR}/Mesh.{FIG_EXT}')

### Make a structured basis


In [ ]:
def make_mass_matrix(function_space: dfn.FunctionSpace):
    _u = dfn.TrialFunction(function_space)
    _du = dfn.TestFunction(function_space)

    # form = _u*_du*dfn.Measure('dx', function_space.mesh())
    form = _u*_du*dfn.Measure('dx')
    return dfn.assemble(form, tensor=dfn.PETScMatrix())

def make_structured_basis_vectors(
        function_space: dfn.FunctionSpace, mesh: dfn.Mesh,
        cell_func: dfn.MeshFunction, cell_label_to_id: Mapping[str, int]
    ):
    """
    Return a dictionary of structured matrix
    """

    cell_label_to_dofs = process_meshlabel_to_dofs(
        mesh, cell_func, cell_label_to_id, function_space.dofmap()
    )
    xdofs = function_space.tabulate_dof_coordinates()[:, 0]
    dofs_body = cell_label_to_dofs['body']
    dofs_cover = cell_label_to_dofs['cover']

    ## Create different structured changes:
    basis_keys = ['uniform', 'body-cover-diff', 'cover-is-grad', 'body-is-grad']
    basis_coeffvecs = {
        key: dfn.Function(function_space).vector()
        for key in basis_keys
    }
    for vec in basis_coeffvecs.values():
        vec[:] = 0

    # uniform change
    basis_coeffvecs['uniform'][:] = 1

    # body/cover change
    basis_coeffvecs['body-cover-diff'][dofs_body] = 1
    basis_coeffvecs['body-cover-diff'][dofs_cover] = -1

    # cover inf-sup gradient change (superior > inferior)
    basis_coeffvecs['cover-is-grad'][dofs_cover] = -1 * xdofs[dofs_cover]

    # body inf-sup gradient change (superior > inferior)
    basis_coeffvecs['body-is-grad'][dofs_body] = -1 * xdofs[dofs_body]

    return basis_coeffvecs

def make_structured_basis_matrix(
        function_space: dfn.FunctionSpace, mesh: dfn.Mesh,
        cell_func: dfn.MeshFunction, cell_label_to_id: Mapping[str, int],
        basis_name: str='BC'
    ):
    """
    Return a basis matrix
    """

    ## Create stiffness perturbations to represent different structured
    ## stiffness changes:
    basis_vecs = make_structured_basis_vectors(
        function_space, mesh, cell_func, cell_label_to_id
    )

    ## Orthonormalize the basis functions
    M = make_mass_matrix(function_space)
    def inner(x, y):
        return x.inner(M*y)

    def orth_proj(x, y):
        y2norm = y.norm('l2')**2
        return inner(x, y)/inner(y, y) * y

    def orthogonalize(x, *ys):
        res = x
        for y in ys:
            res = res - orth_proj(res, y)
        return res
        # return functools.reduce(lambda a, b: a - orth_proj(a, b), ys, x)

    def normalize(x):
        return x/inner(x, x)**0.5

    basis_keys = list(basis_vecs.keys())
    for n, basis_key in enumerate(basis_keys):
        # Succesively orthogonalize each basis vector against previous vectors
        basis_vec = basis_vecs[basis_key]
        basis_vecs[basis_key] = orthogonalize(
            basis_vec, *[basis_vecs[key] for key in basis_keys[:n]]
        )

    for basis_key, basis_vec in basis_vecs.items():
        basis_vecs[basis_key] = normalize(basis_vec)

    if basis_name == 'BC':
        return form_basis([basis_vecs[key] for key in ['uniform', 'body-cover-diff']])
    elif basis_name == 'BC_CINFSUP':
        return form_basis([basis_vecs[key] for key in ['uniform', 'body-cover-diff', 'cover-is-grad']])
    elif basis_name == 'BC_CINFSUP_BINFSUP':
        return form_basis([basis_vecs[key] for key in ['uniform', 'body-cover-diff', 'cover-is-grad', 'body-is-grad']])
    else:
        raise ValueError()

In [ ]:
## Basis matrices

V = HOPF_MODEL.res.solid.residual.form['coeff.prop.emod'].function_space()
mf = HOPF_MODEL.res.solid.residual.mesh_function('cell')
mf_label_to_value = HOPF_MODEL.res.solid.residual.mesh_function_label_to_value('cell')
mesh = HOPF_MODEL.res.solid.residual.mesh()

# Basis with variations in: uniform, body-cover
B_BC = make_structured_basis_matrix(V, mesh, mf, mf_label_to_value, 'BC')

# Basis with variations in: uniform, body-cover, inf-sup in cover
B_BC_CINFSUP = make_structured_basis_matrix(V, mesh, mf, mf_label_to_value, 'BC_CINFSUP')

# Basis with variations in: uniform, body-cover, inf-sup in cover, and inf-sup in body
B_BC_CINFSUP_BINFSUP = make_structured_basis_matrix(V, mesh, mf, mf_label_to_value, 'BC_CINFSUP_BINFSUP')

In [ ]:
## Test the basis is orthonormal
M = make_mass_matrix(V)
print(B_BC_CINFSUP_BINFSUP.T @ M.mat()[:, :] @ B_BC_CINFSUP_BINFSUP)

In [ ]:
# Plot basis vectors representing layered stiffnesses

fig = plt.figure(figsize=(FIG_LX, 0.8*FIG_LY), constrained_layout=True)

basis_vecs = make_structured_basis_vectors(V, mesh, mf, mf_label_to_value)
N = len(basis_vecs)
gs = mpl.gridspec.GridSpec(
    1 + 2, N//2, height_ratios=[0.05]+2*[1],
    figure=fig
)
ax_cbar = fig.add_subplot(gs[0, :])
axs = np.array([
    [fig.add_subplot(gs[row, col]) for col in range(gs.ncols)]
    for row in range(1, gs.nrows)
])

artists = plot_triplots(
    fig, axs.flat,
    [MESH.coordinates()]*N, [MESH.cells()]*N, list(basis_vecs.values()),
    lw=0
)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel("$B$ [kPa]")

format_xaxis_top(ax_cbar.xaxis)

for ax in axs.flat:
    ax.set_aspect(1)

format_axes_grid(axs, "x [cm]", "y [cm]")

descrs = [
    "uniform", "body-cover",
    "IS gradient" "\n" "in cover",
    "IS gradient" "\n" "in body"
]
eigvec_labels = [
    f"$B_{{ {i+1} }}$: {descr}"
    for i, descr in enumerate(descrs)
]
# print(eigvec_labels)
for ax, eigvec_label in zip(axs.flat, eigvec_labels):
    ax.text(0, 1, eigvec_label, transform=ax.transAxes, va='bottom')

fig.savefig(f"{FIG_DIR}/LayeredBasis.{FIG_EXT}")

### Onset pressure sensitivity illustration (for defence)

In [ ]:
coords = MESH.coordinates()
cells = MESH.cells()

tri = mpl.tri.Triangulation(*coords.T, cells)

In [ ]:
xbary = np.array([
    1/3*np.sum(np.array(cell.get_vertex_coordinates()).reshape(-1, 2), axis=0)
    for cell in dfn.cells(MESH)
])

In [ ]:
perturbation_name = 'infsup'
alpha_min, alpha_max = -4, 10
for alpha in [2, 4, 6, 8]:
    fig = plt.figure(
        figsize=(FIG_LX_WIDE, FIG_LY), constrained_layout=True
    )

    gs = gridspec.GridSpec(3, 3, width_ratios=[1, 0.05, 1], figure=fig)

    ax_lin = fig.add_subplot(gs[0, 0])
    ax_lin_cbar = fig.add_subplot(gs[0, 1])
    ax_del = fig.add_subplot(gs[1, 0])
    ax_del_cbar = fig.add_subplot(gs[1, 1])
    ax_sum = fig.add_subplot(gs[2, 0])
    ax_sum_cbar = fig.add_subplot(gs[2, 1])
    axs_mesh = np.array([ax_lin, ax_del, ax_sum])
    axs_mesh_cbar = np.array([ax_lin_cbar, ax_del_cbar, ax_sum_cbar])

    format_axes_grid(axs_mesh[:, None], "x [cm]", "y [cm]")
    for ax in axs_mesh:
        ax.set_aspect(1)

    ax_pon = fig.add_subplot(gs[:, -1])

    ## Plot the linearization, delta, and final stiffness
    z_lin = 6*np.ones(cells.shape[0])

    if perturbation_name == 'linspace':
        z_del = np.linspace(0, 1, cells.shape[0])
        lambda_quad = 2
        lambda_line = 2
    elif perturbation_name == 'infsup':
        z_del = xbary[:, 0]/np.max(xbary[:, 0])
        lambda_quad = 5
        lambda_line = 2
    elif perturbation_name == 'medlat':
        z_del = xbary[:, 1]/np.max(xbary[:, 1])
        lambda_quad = 2
        lambda_line = 10

    # alpha = 1
    z_sum = z_lin + alpha*z_del

    alphas = np.linspace(alpha_min, alpha_max, 101)

    zs = np.array([z_lin, z_del, z_sum])
    zlims = [(5, 8), (-1.5, 1.5), (3, 12)]
    for z, zlim, ax, ax_cbar in zip(zs, zlims, axs_mesh, axs_mesh_cbar):
        artist = ax.tripcolor(coords[:, 0], coords[:, 1], z, triangles=cells, vmin=zlim[0], vmax=zlim[1])
        fig.colorbar(artist, cax=ax_cbar)

    labels = ["$E \, [\mathrm{kPa}]$", "$\Delta E \, [\mathrm{kPa}]$", "$E \, [\mathrm{kPa}]$"]
    for ax, label in zip(axs_mesh_cbar, labels):
        ax.set_ylabel(label)
        ax.yaxis.set_tick_params(left=True, labelleft=True, right=False, labelright=False)

    ## Add the +/= signs and labels
    labels = [f"$+$", "$=$"]
    for ax, label in zip([ax_del, ax_sum], labels):
        ax.text(
            -0.5, 0.5, label,
            ha='right', va='center', fontsize=20,
            transform=ax.transAxes
        )

    labels = ["$E_0$", f"${alpha:+.1f}\Delta E$", f"$E_0 {alpha:+.1f} \\Delta E$"]
    for ax, label in zip(axs_mesh, labels):
        ax.text(
            0.05, 0.95, label,
            ha='left', va='top',
            transform=ax.transAxes
        )

    ## Plot the onset pressure
    pons = 500 + lambda_line*alphas + lambda_quad*alphas**2
    ax_pon.plot(alphas, pons)

    xticks = np.arange(alpha_min, alpha_max+1, 2)
    xticklabels = [
        f"${x:+.1f}\\Delta E$" if x !=0 else "$E_0$"
        for x in xticks
    ]
    ax_pon.set_xticks(xticks)
    ax_pon.set_xticklabels(xticklabels, rotation=-45)
    ax_pon.set_ylabel("$p_\mathrm{on}$ $[\mathrm{Pa}]$")

    ax_pon.axvline(alpha, ls=':', color='k')

    ax_pon.set_ylim(400, 1200)

    fig.savefig(f"{FIG_DIR}/OnsetPressureVariation--{perturbation_name}--{alpha:.1f}.{FIG_EXT}")

### Plot a fixed point / VF static state

In [ ]:
# Solve for a fixed under specified conditions (below)
dyn_model = HOPF_MODEL.res

control = dyn_model.control.copy()

print(control)
control['psub'][:] = 800*10

prop = dyn_model.prop.copy()
prop['emod'][:] = 1e5
prop['ymid'][:] = MESH.coordinates()[:, 1].max() + 1e-1

state_fp, info = libhopf.solve_fp(dyn_model, control, prop)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))

xy = MESH.coordinates() + 1*np.array(state_fp['u'])[VERT_TO_VDOF].reshape(-1, 2)
ax.triplot(xy[:, 0], xy[:, 1], triangles=MESH.cells())

ax.set_aspect(1)
ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")

fig.savefig(f'{FIG_DIR}/DeformedMesh.{FIG_EXT}')

### Plot schematic Taylor models

Plot different Taylor model contours to explain what the principal values mean

In [ ]:
if FIG_STYLE == 'presentation':
    fig = plt.figure(figsize=(FIG_LX_WIDE/2, FIG_LY), constrained_layout=True)
    gs = gridspec.GridSpec(1+2, 2, height_ratios=[0.05, 1, 1], figure=fig)
    # axs = np.array([
    #     [fig.add_subplot(gs[i, j]) for j in range(gs.ncols)]
    #     for i in range(1, gs.nrows)
    # ])
    axs = np.array([[fig.add_subplot(gs[1:, :])]])
    ax_cbar = fig.add_subplot(gs[0, :])
else:
    fig = plt.figure(figsize=(FIG_LX, 0.9*FIG_LY), constrained_layout=True)
    gs = gridspec.GridSpec(1+2, 2, height_ratios=[0.05, 1, 1], figure=fig)
    axs = np.array([
        [fig.add_subplot(gs[i, j]) for j in range(gs.ncols)]
        for i in range(1, gs.nrows)
    ])
    ax_cbar = fig.add_subplot(gs[0, :])

# Set a list of eigenvalue cases (`As`) and the eigenvectors `Z`
As = np.array([
    np.diag([0.1, 2]), np.diag([2, -1]),
    np.diag([2, 0.001]), np.diag([-2, 0.001])
])
Z = np.stack([[1, -1], [1, 1]], axis=-1)
Z = Z/np.diag(Z.T@Z)**0.5
Hs = np.array([Z @ A @ Z.T for A in As])
Gs = np.array([np.array([0, 0])]*3 + [2*np.array([1, 1])])

ec_min, eb_min = 3, 5
ec = np.linspace(-ec_min, 10, 100) + ec_min
eb = np.linspace(-eb_min, 10, 50) + eb_min

EC, EB = np.meshgrid(ec-ec_min, eb-eb_min, indexing='ij')
E = np.stack([EB, EC], axis=-1)

pons = [
    np.sum((E @ hess)*E, axis=-1) + np.dot(E, grad) + 250
    for hess, grad in zip(Hs, Gs)
]
vmin = np.min([np.min(pon) for pon in pons])
vmax = np.max([np.max(pon) for pon in pons])

levels = np.linspace(vmin, vmax, 13)
artists = [
    ax.contourf(eb, ec, pon, cmap=mpl.cm.viridis, vmin=vmin, vmax=vmax, levels=levels)
    for ax, pon in zip(axs.flat, pons)
]

gradlabel_offsets = [
    (-1, -2), (-3, 1), (0, 1), (0, 0.5)
]
linlabel_offsets = [
    (-2, 0), (1, 1), (0.5, 0), (-1.0, 0.5)
]
arrow_kwargs = {
    'width': 0.1,
    'head_width': 0.3,
    'fc': 'k'
}

# Eigenvalues indices
idxs = [(2, 1), (1, 2), (1, 2), (1, 0)]
for ax, A, hess, grad, gradlabel_offset, linlabel_offset, idx in zip(axs.flat, As, Hs, Gs, gradlabel_offsets, linlabel_offsets, idxs):
    # Annotate the Hessian/eigenvalues and critical point
    E_min = np.array([eb_min, ec_min])

    if FIG_STYLE == 'presentation':
        # ax.annotate(f"$E*$", E_min+(-0.5, 0), ha='right', color='red')
        # ax.annotate(f"$E*$", E_min, ha='right')
        # ax.plot([E_min[0]], [E_min[1]], marker='o', color='k', ms=3)
        pass
    else:
        ax.annotate(f"$E*$", E_min, ha='right')
        ax.arrow(*E_min, *Z[:, 0], **arrow_kwargs)
        ax.arrow(*E_min, *Z[:, 1], **arrow_kwargs)

    if FIG_STYLE == 'presentation':
        # ax.annotate(r"${d^2 p_\mathrm{on}}/{dE^2}$", (E_min+1.5*Z[:, 0]))
        # ax.annotate(f"$\\Lambda_{{{idx[0]:d}}}={np.diag(A)[0]:.1f}$", (E_min+1.5*Z[:, 0]))
        # ax.annotate(f"$\\Lambda_{{{idx[1]:d}}}={np.diag(A)[1]:.1f}$", (E_min+1.5*Z[:, 1]))
        pass
    else:
        ax.annotate(f"$\\Lambda={np.diag(A)[0]:.1f}$", (E_min+1.5*Z[:, 0]))
        ax.annotate(f"$\\Lambda={np.diag(A)[1]:.1f}$", (E_min+1.5*Z[:, 1]))

    # Annotate the linearization point
    ec_0, eb_0 = 7, 7
    E_0 = np.array([eb_0, ec_0])
    if FIG_STYLE == 'presentation':
        ax.annotate(f"$E_0$", E_0+linlabel_offset)
        ax.plot([E_0[0]], [E_0[1]], marker='o', color='k', ms=3)

        ax.arrow(*E_0, *Z[:, 0], **arrow_kwargs)
        ax.arrow(*E_0, *Z[:, 1], **arrow_kwargs)
        pass
    else:
        ax.annotate(f"$E_0$", E_0+linlabel_offset)

    # Annotate the gradient
    dE = 0.5*(hess@(E_0-E_min) + grad)
    if FIG_STYLE == 'presentation':
        pass
    else:
        ax.arrow(*E_0, *dE, **arrow_kwargs)
        xy = (E_0+dE)
        xytext = (E_0+dE+gradlabel_offset)
        ax.annotate(r"$d{p_\mathrm{on}}/dE$", xy, xytext=xytext)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.xaxis.set_tick_params(rotation=-45)
ax_cbar.set_xlabel("$p_\mathrm{on}$ [Pa]")
format_xaxis_top(ax_cbar.xaxis)

format_axes_grid(axs, "$E_b$ [kPa]", "$E_c$ [kPa]")

for ax, alph in zip(axs.flat, 'ABCD'):
    # ax.annotate(
    #     f"{alph}", (0.05, 1.05), xycoords=ax.transAxes,
    #     va='bottom', ha='left'
    # )
    ax.set_aspect(1)
    ax.set_xlim(1)
    ax.set_ylim(1)

fig.savefig(f"{FIG_DIR}/SchematicTaylorModel.{FIG_EXT}")

### Sensitivity results for a single linearization point

#### Define the base (linearization) point/stiffness

In [ ]:
## Load results from the given linearization point
mesh_name = 'M5_BC--GA3.00--DZ1.50e+00--NZ12--CL9.40e-01'
# mesh_name = 'M5_CB_GA3_CL0.50'

BIFURCATION_PARAM = 'psub'
# BIFURCATION_PARAM = 'qsub'
    
FUNCTIONAL_NAME = 'OnsetPressure'
# FUNCTIONAL_NAME = 'SubglottalPressure'

PARAM_OPTION = 'Stiffness'
PARAM_OPTION = 'TractionShape'

SENSITIVITY_PARAM = 'tmesh'

# emod_cov, emod_bod = 1/2*6.0e4, 6.0e4
# emod_cov, emod_bod = 1/3*6.0e4, 6.0e4
# emod_cov, emod_bod = 1/4*6.0e4, 6.0e4

emod = 6e4
emod_cov, emod_bod = emod, emod

params_pon = {
    'MeshName': mesh_name,
    'Ecov': emod_cov, 'Ebod': emod_bod,
    'Functional': FUNCTIONAL_NAME,
    'ParamOption': PARAM_OPTION,
    'BifParam': BIFURCATION_PARAM,
    'H': 1e-3
}

params_fon = params_pon.copy()
params_fon['Functional'] = 'OnsetFrequency'

In [ ]:
param = DEFAULT_PARAM.substitute(params_pon)
hopf_model = load_hopf_model(param)

In [ ]:
prop_base = load_block_array(params_pon, 'prop')[0]
transform, scale = setup_transform(
    params_pon, hopf_model, prop_base
)
param_base = load_block_array(params_pon, 'param')[0]

xhopf_base = load_block_array(params_pon, 'state')[0]

# Load eigenpairs of the Hessian, then sort them
eigvals, eigvecs = load_eigenpair(params_pon)


# Flip stiffness eigenvectors to have consistent positive correlations with the
# body-cover basis
if 'emod' in eigvecs[0]:
    eigvecs_emod = [vec['emod'] for vec in eigvecs]
    for eigvec, eigvec_emod in zip(eigvecs, eigvecs_emod):
        eigvec['emod'][:] = np.sign(np.dot(eigvec_emod, B_BC[:, 1]))*eigvec_emod

grad_pon = load_block_array(params_pon, 'grad_param')[0].sub[SENSITIVITY_PARAM]
# grad_fon = load_block_array(params_fon, 'grad_param')[0].sub[SENSITIVITY_PARAM]

#### Plot traction-based shape eigenvectors

In [ ]:
num_eig = 3

sl_residual = hopf_model.res.solid.residual
func_space = sl_residual.form['coeff.state.u1'].function_space()
coords = func_space.tabulate_dof_coordinates()

mesh = sl_residual.mesh()
mesh_coords = mesh.coordinates()
ndim = mesh.topology().dim()

vert_to_vdof = dfn.vertex_to_dof_map(func_space)

In [ ]:
print(eigvecs[0]['tmesh'])

print(eigvals)

In [ ]:
u_meshes = [
    mesh_traction_to_displacement(transform, eigvec['tmesh'])[vert_to_vdof].reshape(-1, ndim) 
    for eigvec in eigvecs
]

In [ ]:
import pyvista as pv
from pyvista import CellType

# `cells` is the `pyvista` mesh format;
# the first entry in the cell is the number of points in the cell (4 for tetrahedron)
mesh_cells = mesh.cells()
cells = np.zeros((mesh_cells.shape[0], mesh_cells.shape[1]+1), dtype=int)
cells[:, 1:] = mesh_cells
cells[:, 0] = 4

cell_types = CellType.TETRA*np.ones(cells.shape[0], dtype=int)

points = mesh.coordinates()

In [ ]:
pv.start_xvfb()
plotter = pv.Plotter(shape=(1, num_eig), border=True, window_size=(2048, 400))

for n_eig, u_mesh in enumerate(u_meshes[:num_eig]):
    plotter.subplot(0, n_eig)
    grid = pv.UnstructuredGrid(cells, cell_types, points+2.0*u_mesh)
    plotter.add_mesh(grid, show_edges=True)

plotter.link_views()

plotter.camera_position = [0, 0.5, 1.5]
plotter.camera.focal_point = [0, 0, 0]

plotter.view_xy()
    
plotter.show()

In [ ]:
tri = mpl.tri.Triangulation(
    mesh_coords[:, 0], mesh_coords[:, 1], mesh.cells()
)

fig, axs = plt.subplots(1, num_eig+1, figsize=(15, 5))
axs = axs.reshape((1, num_eig+1))
axs_eig = axs[:, :-1]
ax_min = axs[0, -1]

for ax, eigvec in zip(axs_eig.flat, eigvecs[:num_eig]):
    x, y = coords[::2, 0], coords[::2, 1]
    tx, ty = eigvec['tmesh'][:-1:2], eigvec['tmesh'][1::2]
    # Scale the arrows so the longest one has a length of 0.1
    scale = 0.1 / np.max((tx**2+tx**2)**0.5)
    for _x, _y, _tx, _ty in zip(x, y, tx, ty):
        ax.arrow(_x, _y, scale*_tx, scale*_ty)

    # Plot the original mesh
    ax.triplot(*(mesh_coords).T, mesh.cells(), alpha=0.5, color='k', lw=0.3)

    # Compute/plot the mesh motion corresponding to the traction
    umesh = mesh_traction_to_displacement(transform, eigvec['tmesh'])
    umesh = umesh[VERT_TO_VDOF].reshape(-1, 2)
    ax.triplot(*(mesh_coords+0.5*umesh).T, mesh.cells(), lw=0.5)

# Plot the shape that minimizes onset pressure
num_eig_for_min = 3
da, dtmesh = solve_low_rank_eig(
    eigvals[:num_eig_for_min], 
    form_basis([e.sub['tmesh'][:] for e in eigvecs[:num_eig_for_min]]),
    -grad_pon
)
umesh = mesh_traction_to_displacement(transform, dtmesh)
umesh = umesh[VERT_TO_VDOF].reshape(-1, 2)
ax_min.triplot(*(mesh_coords+umesh).T, mesh.cells(), lw=0.5)

for ax in axs.flat:
    ax.set_aspect(1)

for ax, eigval in zip(axs_eig.flat, eigvals):
    ax.text(
        0.05, 0.95, f"$\\lambda=$ {eigval:.1e}",
        va='top', transform=ax.transAxes
    )

format_axes_grid(axs, "x [cm]", "y [cm]")
print(np.dot(eigvecs[0]['tmesh'], eigvecs[2]['tmesh']))

fig.savefig(f'{FIG_DIR}/TractionShapeEigenvectors.{FIG_EXT}')

#### Plot mesh displacement-based shape eigenvectors

In [ ]:
num_eig = 4

sl_residual = HOPF_MODEL.res.solid.residual
func_space = sl_residual.form['coeff.state.u1'].function_space()
coords = func_space.tabulate_dof_coordinates()

mesh = sl_residual.mesh()
mesh_coords = mesh.coordinates()
tri = mpl.tri.Triangulation(
    mesh_coords[:, 0], mesh_coords[:, 1], mesh.cells()
)

fig, axs = plt.subplots(1, num_eig, figsize=(5, 3))
axs = axs.reshape((1, num_eig))

for ax, eigvec in zip(axs.flat, eigvecs[:num_eig]):

    # Plot the original mesh
    ax.triplot(*(mesh_coords).T, mesh.cells(), alpha=0.5, color='k', lw=0.3)

    # Compute/plot the mesh motion corresponding to the traction
    umesh = eigvec['umesh'][VERT_TO_VDOF].reshape(-1, 2)
    scale = 0.5*np.max(np.linalg.norm(umesh, axis=-1))
    ax.triplot(*(mesh_coords+scale*umesh).T, mesh.cells(), lw=0.5)

for ax in axs.flat:
    ax.set_aspect(1)

for ax, eigval in zip(axs.flat, eigvals):
    ax.text(
        0.05, 0.95, f"$\\lambda=$ {eigval:.1f}",
        va='top', transform=ax.transAxes
    )

format_axes_grid(axs, "x [cm]", "y [cm]")
# print(np.dot(eigvecs[0]['umesh'], eigvecs[0]['umesh']))

fig.savefig(f'{FIG_DIR}/ShapeEigenvectors.{FIG_EXT}')

#### Form and plot structured stiffness bases

In [ ]:
## Compute eigenpairs of the Hessian in an apriori structured basis
STRUCT_BASIS_TYPE = 'BC_INFSUPCOV_INFSUPBOD'

if STRUCT_BASIS_TYPE == 'BC':
    B_BASIS = B_BC
elif STRUCT_BASIS_TYPE == 'BC_INFSUPCOV':
    B_BASIS = B_BC_CINFSUP
elif STRUCT_BASIS_TYPE == 'BC_INFSUPCOV_INFSUPBOD':
    B_BASIS = B_BC_CINFSUP_BINFSUP
else:
    raise ValueError(f"Unknown `BASIS_TYPE` {STRUCT_BASIS_TYPE}")

Z_EIG = form_basis([eigvec['emod'] for eigvec in eigvecs])
PROJ = form_projector(B_BASIS)

subspace_eigdecomp = np.linalg.eigh(
    np.dot(PROJ.T, Z_EIG) @ np.diag(eigvals) @ np.dot(Z_EIG.T, PROJ)
)
idx_sort = argsort(subspace_eigdecomp[0], sort_by='real')[::-1]

# cset_grads = [proj@grad_pon for proj in projectors]

proj_eigvals = subspace_eigdecomp[0][idx_sort]
# This line ensure `proj_eigvecs` is a list containing each eigenvector
proj_eigvecs = [subspace_eigdecomp[1][:, ii] for ii in idx_sort]

# Flip eigenvectors to have consistent positive correlations with the body-cover difference
proj_eigvecs = [np.sign(np.dot(vec, B_BC[:, 1]))*vec for vec in proj_eigvecs]

proj_grad_f = PROJ@grad_pon

#### Plot stiffness eigenvectors/conceptual quadratic model

In [ ]:
## Plot the eigenvectors and gradient for debugging
num_eig = 5
fig, _axs = plt.subplots(1, 1+num_eig)

ax_grad = _axs[0]
axs = _axs[1:]

coords = MESH.coordinates()
cells = MESH.cells()
tri = mpl.tri.Triangulation(*coords.T, cells)

plot_triplots(fig, [ax_grad], [coords], [cells], [grad_pon])

plot_triplots(fig, axs, num_eig*[coords], num_eig*[cells], [x['emod'] for x in eigvecs[:num_eig]])
for ax, eigval in zip(axs, eigvals):
    ax.text(0, 1.05, f"$\\lambda={eigval:.1f}$", transform=ax.transAxes)

for ax in [ax_grad] + list(axs):
    ax.set_aspect(1)

format_axes_grid(np.atleast_2d(_axs), "x [cm]", "y [cm]")

In [ ]:
## Solve a constrained quadratic minimization problem
# min 1/2 x^T d2f_dx2 x + df_dx x
# 'a' will represent the vector of coefficient in the reduced basis

USE_STRUCT_BASIS = False

# Z_EIG = form_basis([x['emod'] for x in eigvecs])
# eigvals = np.diag(eigvals[:])

## Set the low-dimensional basis for approximation
# 'Z' is the reduced dimensional basis to solve for changes in E, in
# NOTE: For `B_BC_CINFSUP_BINFSUP`, using the uniform basis component results in very negative
# predictions for the minimum onset pressure

# Pick the first two eigenpairs to form a low-dimensional basis for the quadratic model
# Usually the first two, sorted in descending order, have positive real eigval and
# smooth, physically meaningful, eigenvectors
if USE_STRUCT_BASIS:
    r_eigvecs = proj_eigvecs[:1]
    r_eigvals = proj_eigvals[:1]
elif BIFURCATION_PARAM == 'qsub':
    r_eigvecs = [x['emod'] for x in eigvecs[:1]]
    r_eigvals = eigvals[:1]
else:
    r_eigvecs = [x['emod'] for x in eigvecs[:2]]
    r_eigvals = eigvals[:2]

da, demod = solve_low_rank_eig(
    r_eigvals, form_basis(r_eigvecs), -grad_pon
)

In [ ]:
# Use the parameterization to map the change in parameters to a change in base properties
delta_param_min = param_base.copy()
delta_param_min[:] = 0
delta_param_min['emod'] = demod
param_min = param_base + delta_param_min
# delta_props_min = TRANSFORM.apply_jvp(param_base, delta_param_min)
prop_min = transform.apply(param_min)

In [ ]:
## Compute the onset condition (Hopf bif.) at the linearization and critical points

# xhopf_min_n = xhopf
xhopf_min, info = libhopf.solve_hopf_by_newton(HOPF_MODEL, xhopf_base, prop_min)

fonsets = 1/(2*np.pi)*np.array(
    [x.sub['omega'][0] for x in (xhopf_base, xhopf_min)]
)
ponsets = 1/10*np.array(
    [x.sub[BIFURCATION_PARAM][0] for x in (xhopf_base, xhopf_min)]
)

print(ponsets)

In [ ]:
# This is the minimum onset pressure as predicted by the quadratic model
# (not the actual onset pressure at the critical point)
Z = form_basis(r_eigvecs)
if FUNCTIONAL_NAME == 'OnsetPressure':
    ponset_critical = xhopf_base.sub['psub'][0] + 1/2*grad_pon @ Z @ da
    print(ponset_critical, xhopf_base.sub['psub'][0])
elif FUNCTIONAL_NAME == 'SubglottalPressure':
    ponset_critical = xhopf_base.sub['p'][0] + 1/2*grad_pon @ Z @ da
    print(ponset_critical, xhopf_base.sub['p'][0])
else:
    raise ValueError("")

##### Plot minimizer

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, 10*INCH))

grid_spec = mpl.gridspec.GridSpec(2, 2, figure=fig, height_ratios=[0.025, 1])
ax_cbar = fig.add_subplot(grid_spec[0, :])
axs = np.array([fig.add_subplot(grid_spec[1, n]) for n in range(grid_spec.ncols)])

zs = [x.sub['emod']/10/1e3 for x in (prop_base, prop_min)]
coords = MESH.coordinates()
cells = MESH.cells()
artists = plot_triplots(fig, axs, len(zs)*[coords], len(zs)*[cells], zs, lw=0)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, fonset, ponset in zip(axs.flat, fonsets, ponsets):
    text = "$(f, p)_\mathrm{onset}=$" + f"({fonset:.1f} Hz, {ponset/1:.1f} Pa)"
    ax.text(0, 1, text, transform=ax.transAxes, ha='left', va='top')

ax_cbar.set_xlabel("$E$ [kPa]")
format_xaxis_top(ax_cbar.xaxis)

axs[1].yaxis.set_tick_params(labelleft=False)

labels = ["Original distribution", "Onset pressure minimizer"]
for ax, label in zip(axs, labels):
    ax.text(0, 1, label, transform=ax.transAxes, ha='left', va='bottom')

for ax in axs:
    ax.set_aspect(1)
    ax.set_xlabel("x [cm]")

axs[0].set_ylabel("y [cm]")

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/QuadraticApproximateMinimizer.{FIG_EXT}')

##### Plot 3D conceptual quadratic model

In [ ]:
# Z = form_basis(eigvecs[:2])
# `Z` is the low-dimensional basis and is set in earlier cells when solving for the critical point

# Let `r_grad` denote the gradient with quadratic eigenvector components removed
if USE_STRUCT_BASIS:
    r_grad = proj_grad_f - Z@(proj_grad_f@Z)
else:
    r_grad = grad_pon - Z@(grad_pon@Z)

# This finds the component of the gradient not along eigenvectors
grad_magnitude = np.linalg.norm(r_grad)
grad_unit = r_grad/grad_magnitude

In [ ]:
PLOT_FON = False
# This is the distance travelled along the linear/gradient component
a0 = 0

# Dimension of the low-dimensional subspace (for quadratic effects)
NUM_EIG = len(r_eigvals)
print(NUM_EIG)

## 3 column layout
if FIG_STYLE == 'presentation':
    fig = plt.figure(figsize=(FIG_LX_WIDE, 1.0*FIG_LY), constrained_layout=True)
else:
    if NUM_EIG >= 2:
        fig = plt.figure(figsize=(FIG_LX_WIDE, 1.0*FIG_LY), constrained_layout=True)
    else:
        fig = plt.figure(figsize=(FIG_LX_WIDE, 0.8*FIG_LY), constrained_layout=True)

# The `NUM_EIG+1` is to plot each eigenvector +  the linear effect
gs = mpl.gridspec.GridSpec(
    1+(1+NUM_EIG), 3, figure=fig,
    width_ratios=[1, 1, 1], height_ratios=[0.1]+(NUM_EIG+1)*[1], hspace=0
)

# Quadratic contours are in column 0
ax_main = fig.add_subplot(gs[1:, 0])
ax_main_cbar = fig.add_subplot(gs[0, 0])

# Eigenvectors/gradient are in column 1
axs_modes = np.array([fig.add_subplot(gs[ii, 1]) for ii in range(1, gs.nrows)])
ax_modes_cbar = fig.add_subplot(gs[0, 1])

# Absolute stiffness (initial and critical points) are in column 2
ax_base = fig.add_subplot(gs[-2, 2])
ax_min = fig.add_subplot(gs[-1, 2])
ax_point_cbar = fig.add_subplot(gs[0, 2])

## 2 column layout Version A

## Set x/y labels
format_axes_grid(axs_modes[:, None], "x [cm]", "y [cm]")
format_axes_grid(np.array([ax_base, ax_min])[:, None], "x [cm]", "y [cm]")
for ax in list(axs_modes) + [ax_min, ax_base]:
    ax.set_aspect(1)

# axs_modes[-1].set_xlabel("x [cm]")
# ax_min.set_xlabel("x [cm]")

## Plot onset pressure contours
a1 = np.linspace(-15, 15)
a2 = np.linspace(-15, 15)
a1grid, a2grid = np.meshgrid(a1, a2)

# The two sums below use a1, a2 to plot eigenvector/quadratic terms first.
# If there is only one quadratic term, then the linear effect is plotted in the
# last axis (a2).
_x = a1grid[..., None]*r_eigvecs[0][None, :] + a2grid[..., None]*r_eigvecs[1][None, :]
ponset = (
    ponset_critical + np.dot(_x, r_grad)
    + 1/2*inner_eig(_x, _x, r_eigvals, form_basis(r_eigvecs))
)

if not PLOT_FON:
    artist = ax_main.contourf(
        a1, a2, ponset/10, vmin=ponset_critical/10
    )

    fig.colorbar(artist, cax=ax_main_cbar, orientation='horizontal')
    ax_main_cbar.set_xlabel("$p_\mathrm{on}$ [Pa]")
    ax_main_cbar.xaxis.set_tick_params(rotation=-35)
else:
    fonset = (xhopf_base['omega'][0] + np.dot(_x, grad_fon)) / (2*np.pi)

    artist = ax_main.contourf(a1, a2, fonset)

    fig.colorbar(artist, cax=ax_main_cbar, orientation='horizontal')
    ax_main_cbar.set_xlabel("$f_\mathrm{on}$ [Hz]")
    ax_main_cbar.xaxis.set_tick_params(rotation=-35)

# Annotate the initial point on the quadratic contours
# `da` is the step in the eigenbasis from the linearization point to the minimum
if da.size >= 2:
    ax_main.plot(-da[[0]], -da[[1]], marker='o', mfc='w', mec='none')
    ax_main.annotate("$E_0$", -da+np.array([0.5, 0.5]), color='w')
else:
    ax_main.plot(-da[[0]], [0], marker='o', mfc='w', mec='none')
    ax_main.annotate("$E_0$", [-da[0], 0]+np.array([0.5, 0.5]), color='w')

# Annotate the distance along the linear trend on the quadratic contours
# ax_main.annotate(f"$+{a0:0>4.1f} \\Delta E_0$", (1, 1), xycoords=ax_main.transAxes)

ax_main.set_xlim(-15, 15)
ax_main.set_ylim(-15, 15)
ax_main.set_aspect(1)

aticks = np.arange(-15, 16, 5)
ax_main.set_xticks(aticks)
ax_main.set_yticks(aticks)

# This will label x/y axes with eigenvectors and the y-axis with the linear effect
# if there's only one eigenvector
xy_ticklabel_dir = [f"A_{i:d}" for i in range(1, NUM_EIG+1)] + ["A_0"]
xy_ticklabels = [
    [f"${a:+.1f}{dir_label}$" if a!= 0 else r"$E*$" for a in aticks]
    for dir_label in xy_ticklabel_dir
]
ax_main.set_xticklabels(xy_ticklabels[0], rotation=-45)
ax_main.set_yticklabels(xy_ticklabels[1], rotation=0)

## Plot key stiffness points (linearization and critical point)
# Set up the triangulation object first
coords = MESH.coordinates()
cells = MESH.cells()
tri = mpl.tri.Triangulation(*coords.T, cells)

zs = [prop_base.sub['emod']/10/1e3, prop_min.sub['emod']/10/1e3]
axs = [ax_base, ax_min]
n = len(axs)
artists = plot_triplots(fig, axs, n*[coords], n*[cells], zs)
fig.colorbar(artists[0], cax=ax_point_cbar, orientation='horizontal')

## Plot modes (linear gradient and hessian eigenvectors)
n = len(axs_modes)
zs = [grad_unit] + r_eigvecs
artists = plot_triplots(fig, axs_modes, n*[coords], n*[cells], zs, vmin=-0.2, vmax=0.2)
fig.colorbar(artists[0], cax=ax_modes_cbar, orientation='horizontal')

# This labels the final axis as $\Delta E_0$ if there's only one quadratic
# dimension
labels = (
    [f"$A_0, \\Lambda_0={grad_magnitude:.1f}$"]
    + [
        f"$A_{i+1:d}, \\Lambda_{i+1:d}={eigval:.1f}$"
        for i, eigval in enumerate(r_eigvals)
    ]
)
for ax, label in zip(axs_modes, labels):
    # ax.annotate(label, (0.05, 0.95), ha='left', va='top', xycoords=ax.transAxes)
    if FIG_STYLE == 'presentation':
        pass
        ax.annotate(label, (0.05, 1), ha='left', va='bottom', xycoords=ax.transAxes)
    else:
        ax.annotate(label, (0.05, 1), ha='left', va='bottom', xycoords=ax.transAxes)

labels = ["$E_0$", "$E*$"]
for ax, label in zip([ax_base, ax_min], labels):
    # ax.annotate(label, (0.05, 0.95), ha='left', va='top', xycoords=ax.transAxes)
    ax.annotate(label, (0.05, 1), ha='left', va='bottom', xycoords=ax.transAxes)

ax_modes_cbar.set_xlabel("$A$ [kPa]")
ax_point_cbar.set_xlabel("$E$ [kPa]")

## Format colour bars
axs_cbar = [ax_main_cbar, ax_modes_cbar, ax_point_cbar]
for ax_cbar in axs_cbar:
    format_xaxis_top(ax_cbar.xaxis)

# Add column labels to each upper colour bar
# You have to draw the figure first so the labels have a set position
for label, ax in zip('ABC', axs_cbar):
    ax.text(
        0, -1, f"{label}",
        ha='left', va='top', transform=ax.transAxes
    )

if USE_STRUCT_BASIS:
    fig.savefig(
        f"{FIG_DIR}/OnsetPressureTaylorModel"
        f"--a0{a0:.1f}--Ec{emod_cov/1e4:.1f}--Eb{emod_bod/1e4:.1f}"
        f"--Basis{STRUCT_BASIS_TYPE}.{FIG_EXT}"
    )
else:
    fig.savefig(
        f"{FIG_DIR}/OnsetPressureTaylorModel"
        f"--a0{a0:.1f}--Ec{emod_cov/1e4:.1f}--Eb{emod_bod/1e4:.1f}.{FIG_EXT}"
    )


##### Single eigenvector direction plot

In [ ]:
fig = plt.figure(
    figsize=(FIG_LX_WIDE, FIG_LY), constrained_layout=True
)

gs = gridspec.GridSpec(3, 3, width_ratios=[1, 0.05, 1], figure=fig)

ax_lin = fig.add_subplot(gs[0, 0])
ax_lin_cbar = fig.add_subplot(gs[0, 1])
ax_del = fig.add_subplot(gs[1, 0])
ax_del_cbar = fig.add_subplot(gs[1, 1])
ax_sum = fig.add_subplot(gs[2, 0])
ax_sum_cbar = fig.add_subplot(gs[2, 1])
axs_mesh = np.array([ax_lin, ax_del, ax_sum])
axs_mesh_cbar = np.array([ax_lin_cbar, ax_del_cbar, ax_sum_cbar])
for ax in axs_mesh:
    ax.set_aspect(1)
format_axes_grid(axs_mesh[:, None], "x [cm]", "y [cm]")

coords = MESH.coordinates()
cells = MESH.cells()
tri = mpl.tri.Triangulation(*coords.T, cells)

# Plot onset pressure variation along the direction
idx_eig = -1
if idx_eig == -1:
    emod_dir = grad_unit
    lmbda = grad_magnitude
    a_min = 5
else:
    emod_dir = eigvecs[idx_eig]
    lmbda = eigvals[idx_eig]
    a_min = -np.dot(emod_dir, grad_pon)/lmbda

ax_pon = fig.add_subplot(gs[:, -1])
alphas = np.linspace(-15, 15, 26)
# ponsets = np.array([comp_ponset(alpha*emod_dir) for alpha in alphas])
_x = alphas[:, None] * emod_dir
ponsets = 1/2*inner_eig(_x, _x, eigvals, eigvecs) + grad_pon.dot(_x) + xhopf_base['psub'][0]

fonsets = grad_fon.dot(emod_dir) * alphas
ax_pon.plot(alphas, ponsets/10)
ax_pon_twin = ax_pon.twinx()
ax_pon_twin.plot(alphas, fonsets/2/np.pi, color='red')
ax_pon_twin.set_ylabel(f"$f_\mathrm{{on}}$ $[\mathrm{{Hz}}]$")
ax_pon.set_ylim(250, 600)

ax_pon_twin.yaxis.label.set_color('red')

# Plot the stiffness distributions
emod_lin = prop_base.sub['emod']/10/1e3
emod_del = a_min*emod_dir
emod_sum = emod_lin + emod_del
zs = [emod_lin, emod_del, emod_sum]
zlims = [(5, 8), (-1.5, 1.5), (5, 8)]
for z, zlim, ax, ax_cbar in zip(zs, zlims, axs_mesh, axs_mesh_cbar):
    artist = ax.tripcolor(coords[:, 0], coords[:, 1], z, triangles=cells, vmin=zlim[0], vmax=zlim[1])
    fig.colorbar(artist, cax=ax_cbar)

labels = ["$E \, [\mathrm{kPa}]$", "$\Delta E \, [\mathrm{kPa}]$", "$E \, [\mathrm{kPa}]$"]
for ax, label in zip(axs_mesh_cbar, labels):
    ax.set_ylabel(label)
    ax.yaxis.set_tick_params(left=True, labelleft=True, right=False, labelright=False)

xticks = np.arange(-15, 16, 5)
xticklabels = [
    f"${x:+.1f}\\Delta E$" if x !=0 else "$E_0$"
    for x in xticks
]
ax_pon.set_xticks(xticks)
ax_pon.set_xticklabels(xticklabels, rotation=-45)
ax_pon.set_ylabel("$p_\mathrm{on}$ $[\mathrm{kPa}]$")

ax_pon.axvline(a_min, ls='-.', color='k')
# ax_pon.annotate()

## Add the +/= signs and labels
labels = [f"$+$", "$=$"]
for ax, label in zip([ax_del, ax_sum], labels):
    ax.text(
        -0.5, 0.5, label,
        ha='right', va='center', fontsize=20,
        transform=ax.transAxes
    )

if idx_eig != -1:
    labels = ["$E_0$", f"${a_min:+.1f}\Delta E_{{{idx_eig+1:d}}}$", f"$E^*$"]
else:
    labels = ["$E_0$", f"${a_min:+.1f}\Delta E_{{{idx_eig+1:d}}}$", f"$E_0 {a_min:+.1f} \\Delta E_{{{idx_eig+1:d}}}$"]
for ax, label in zip(axs_mesh, labels):
    ax.text(
        0.05, 0.95, label,
        ha='left', va='top',
        transform=ax.transAxes
    )

fig.savefig(f"{FIG_DIR}/OnsetPressureAlongEigvec--{idx_eig:d}.{FIG_EXT}")

#### Compare sensitivity across different subspaces

In [ ]:
# Form eigendecompositions of the Hessian for each subspace of interest
basis_names = [
    'unstructured',
    'BCCB',
    'BCC',
    'BC'
]

# Bases spanning each subspace
B_EIG = form_basis([x['emod'] for x in eigvecs])
bases = [
    B_EIG,
    B_BC_CINFSUP_BINFSUP,
    B_BC_CINFSUP,
    B_BC
]

projectors = [
    form_projector(Z) for Z in bases
]

subspace_eigdecomps = [
    np.linalg.eigh(
        np.dot(proj.T, B_EIG) @ np.diag(eigvals) @ np.dot(B_EIG.T, proj)
    )
    for proj in projectors
]
idx_sorts = [
    argsort(eigdecomp[0], sort_by='real')[::-1]
    for eigdecomp in subspace_eigdecomps
]
subspace_eigdecomps = [
    (eigdecomp[0][idx_sort], eigdecomp[1][:, idx_sort])
    for eigdecomp, idx_sort in zip(subspace_eigdecomps, idx_sorts)
]

coll_grad = [proj@grad_pon for proj in projectors]
coll_eigvals = [eigdecomp[0] for eigdecomp in subspace_eigdecomps]
coll_eigvecs = [eigdecomp[1] for eigdecomp in subspace_eigdecomps]

# print(coll_eigvecs[0])
# Flip the sign of eigenvectors to make sure they're consistent with those in the 3D quadratic model figure
signs = [np.sign(np.dot(B_BC[:, 1], A[:, :5])) for A in coll_eigvecs]
coll_eigvecs = [sign*A[:, :5] for sign, A in zip(signs, coll_eigvecs)]

In [ ]:
# %debug
# Solve for the minimum/critical onset pressure in each case
num_eigs = [2, 1, 1, 1]

# df_da = grad @ Z

das = [
    solve_low_rank_eig(
        eigvals[:n], eigvecs[:, :n], -grad_pon
    )[0]
    for n, eigvals, eigvecs in zip(num_eigs, coll_eigvals, coll_eigvecs)
]

min_onset_pressures = [
    xhopf_base.sub['psub'][0] + 1/2*grad @ Z[:, :n] @ da
    for n, Z, grad, da in zip(num_eigs, coll_eigvecs, coll_grad, das)
]

coll_grad[0].shape
coll_eigvecs[0].shape

##### Summarize eigenvalues for each subspace in a table

In [ ]:
# This is the number of significant eigenpairs for each basis in `basis_names`
NUM_SIG_EIGPAIR = (2, 1, 1, 1)
col_labels = ['BasisName', 'Eigval1', 'Eigval2', 'MinOnsetPressure']

def make_eigval_series(n, eigvals):
    return eigvals[:n], (2-n)*[np.nan]

eigval_series_set = [
    np.array([
        coll_eigvals[m_basis][n_eig] if n_eig < m_sig else np.nan
        for m_basis, m_sig in enumerate(NUM_SIG_EIGPAIR)
    ])
    for n_eig in range(2)
]

col_series = [
    basis_names,
    *eigval_series_set,
    np.array(min_onset_pressures)/10
]
data = {
    name: series
    for name, series in zip(col_labels, col_series)
}
df = pd.DataFrame(data=data)

df['OnsetPressureDecrease'] = xhopf_base.sub['psub'][0]/10 - df['MinOnsetPressure']

In [ ]:
col_label_to_disp = {
    'BasisName': r"Basis",
    'Eigval1': r"$\Lambda_1$",
    'Eigval2': r"$\Lambda_2$",
    'MinOnsetPressure': r"${p_\mathrm{on}}^* \, [\unit{\Pa}]$",
    'OnsetPressureDecrease':
        r"$p_{\mathrm{on}, 0} - {p_\mathrm{on}}^* \, [\unit{\Pa}]$"
}
styler = df.style
styler.format(na_rep='', precision=1)
styler.format_index(
    formatter=lambda label: col_label_to_disp[label], axis=1
)
styler.hide(axis='index')
print(styler.to_latex(column_format='lrrrr'))

print(xhopf_base.sub['psub'][0]/10)

In [ ]:
## Table of norms for the first eigenvector
# Norm for each columns
def normd2(e):
    return np.linalg.norm(e)

_efunc = dfn.Function(FSPACE_DG0)
def norml2(e):
    _efunc.vector()[:] = e
    return dfn.norm(_efunc, 'l2')

def normdinf(e):
    return np.max(np.abs(e))

col_norms = []

# This is the first eigenvector for each basis
row_eigvecs = [
    eigvecs[:, 0]
    for eigvecs in coll_eigvecs
]
df = pd.DataFrame(data={'BasisName': basis_names})

df['D2norm'] = [normd2(eigvec) for eigvec in row_eigvecs]
df['L2norm'] = [norml2(eigvec) for eigvec in row_eigvecs]
df['Dinfnorm'] = [normdinf(eigvec) for eigvec in row_eigvecs]


In [ ]:
col_label_to_disp = {
    'BasisName': r"Basis",
    'D2norm': r"$||E_1||_{2}$",
    'L2norm': r"$||E_1||_{l_2(\Omega)}$",
    'Dinfnorm': r"$||E_1||_{\infty}$"
}
styler = df.style
styler.format(na_rep='', precision=3)
styler.format_index(
    formatter=lambda label: col_label_to_disp[label],
    axis=1
)
styler.hide(axis='index')
print(styler.to_latex())

In [ ]:
## Table of adjusted eigenvalues for the first eigenvector

# Norm for each columns
def normd2(e):
    return np.linalg.norm(e)

_efunc = dfn.Function(FSPACE_DG0)
def norml2(e):
    _efunc.vector()[:] = e
    return dfn.norm(_efunc, 'l2')

def normdinf(e):
    return np.max(np.abs(e))

col_norms = []

# This is the first eigenvector for each basis
row_eigvecs = [
    eigvecs[:, 0] for eigvecs in coll_eigvecs
]
row_eigvals = [
    eigvals[0] for eigvals in coll_eigvals
]
df = pd.DataFrame(data={'BasisName': basis_names})

df['Lambda1D2norm'] = [
    eigval/normd2(eigvec)**2
    for eigvec, eigval in zip(row_eigvecs, row_eigvals)
]
df['Lambda1L2norm'] = [
    eigval/norml2(eigvec)**2
    for eigvec, eigval in zip(row_eigvecs, row_eigvals)
]
df['Lambda1Dinfnorm'] = [
    eigval/normdinf(eigvec)**2
    for eigvec, eigval in zip(row_eigvecs, row_eigvals)
]

print(df)

In [ ]:
col_label_to_disp = {
    'BasisName': r"Basis",
    'Lambda1D2norm': r"$\Lambda_1 \; (||\cdot||_{2})$",
    'Lambda1L2norm': r"$\Lambda_1 \; (||\cdot||_{l_2(\Omega)})$",
    'Lambda1Dinfnorm': r"$\Lambda_1 \; (||\cdot||_{\infty})$"
}
styler = df.style
styler.format(na_rep='', precision=1)
styler.format_index(
    formatter=lambda label: col_label_to_disp[label],
    axis=1
)
styler.hide(axis='index')
print(styler.to_latex())

##### Onset pressure variation along $A_1$

In [ ]:
## Wide Layout
# fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY), constrained_layout=True)
# gs = mpl.gridspec.GridSpec(
#     3, 3, figure=fig,
#     height_ratios=[0.1] + 2*[1], width_ratios=[2, 1, 1]
# )
# ax_ponset = fig.add_subplot(gs[1:, 0])

# axs_emod = np.array(
#     [[fig.add_subplot(gs[i, j]) for j in range(1, 3)] for i in range(1, 3)]
# )
# ax_emod_cbar = fig.add_subplot(gs[0, 1:])

## Horizontal Layout
fig = plt.figure(figsize=(FIG_LX, 1.2*FIG_LY), constrained_layout=True)
gs = mpl.gridspec.GridSpec(
    4, 2, figure=fig,
    height_ratios=[2] + [0.1] + 2*[1], width_ratios=[1, 1]
)
ax_ponset = fig.add_subplot(gs[0, :])

axs_emod = np.array(
    [[fig.add_subplot(gs[i, j]) for j in range(2)] for i in range(2, 4)]
)
ax_emod_cbar = fig.add_subplot(gs[1, :])

## Plot change in onset pressure along `A[:, 0]` (`$\Delta E_1$`)
a = np.linspace(5, -20)
linestyles = ['-', ':', '-.', '--']
for basis_name, eigvals, A, ls in zip(basis_names, coll_eigvals, coll_eigvecs, linestyles):
    ponset = 1/2*eigvals[0]*a**2 + grad_pon@A[:, 0]*a
    ax_ponset.plot(a, ponset/10, label=basis_name, ls=ls)

ax_ponset.legend()

ax_ponset.set_ylabel("$\Delta p_\mathrm{on}$ $[\mathrm{Pa}]$")

xticks = np.arange(5, -(20+1), -5)
xlabels = [
    f"${x:+.1f} A_1$" if x != 0 else "$E_0$"
    for x in xticks
]
ax_ponset.xaxis.set_ticks(xticks, xlabels, rotation=-45)

## Plot stiffness distributions at `-15 A[:, 0]`
emod_base = param_base.sub['emod']
emods = [emod_base -15*A[:, 0] for A in coll_eigvecs]
artists = plot_triplots(fig, axs_emod.flat, 4*[MESH.coordinates()], 4*[MESH.cells()], emods)

fig.colorbar(artists[0], cax=ax_emod_cbar, orientation='horizontal')
ax_emod_cbar.set_xlabel("$E_0 - 15 A_1$ $[\mathrm{kPa}]$")
format_xaxis_top(ax_emod_cbar.xaxis)

format_axes_grid(axs_emod, "$x$ $[\mathrm{cm}]$", "$y$ $[\mathrm{cm}]$")

for ax, basis_name in zip(axs_emod.flat, basis_names):
    ax.set_aspect(1)
    ax.text(0, 1.05, basis_name, transform=ax.transAxes)

fig.savefig(f"{FIG_DIR}/OnsetPressureAlongDeltaE1.{FIG_EXT}")

##### Compare gradient and hessian in restricted subspaces

In [ ]:
## Plot gradient

fig = plt.figure(
    figsize=(FIG_LX_WIDE, 0.6*FIG_LY), constrained_layout=True
)
gs = gridspec.GridSpec(
    2, len(coll_eigvecs), height_ratios=[0.05, 1], figure=fig
)

axs = np.array([fig.add_subplot(gs[1, n]) for n in range(gs.ncols)])
ax_cbar = fig.add_subplot(gs[0, :])

zs = coll_grad
zs_vert = [project_DG0_to_CG1(z)[VERT_TO_SDOF] for z in zs]
lmbdas = [eigvals[0] for eigvals in coll_eigvals]
artists = plot_triplots(fig, axs, [coords]*len(zs), len(zs)*[cells], zs)
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontour', levels=[0], colors='k')
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontourf')
fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, basis_name, lmbda in zip(axs, basis_names, lmbdas):
    text = f"{basis_name}"
    ax.annotate(text, (0, 1.05), xycoords=ax.transAxes)
    ax.set_aspect(1)

for ax in axs[1:]:
    ax.yaxis.set_tick_params(labelleft=False)

for ax in axs:
    ax.set_xlabel("x [cm]")

axs[0].set_ylabel("y [cm]")

ax_cbar.set_xlabel("${{dp_\\mathrm{on}}} / {{dE}}$ [$\mathrm{Pa}/\mathrm{kPa}$]")
# fig.tight_layout()

fig.savefig(f"{FIG_DIR}/GradientComparisonAcrossSubspaces.{FIG_EXT}")

In [ ]:
## Plot gradient without principal sensitivity component

fig = plt.figure(figsize=(FIG_LX_WIDE, 0.65*FIG_LY), constrained_layout=True)
gs = gridspec.GridSpec(2, len(coll_eigvecs), height_ratios=[0.05, 1], figure=fig)

axs = np.array([fig.add_subplot(gs[1, n]) for n in range(gs.ncols)])
ax_cbar = fig.add_subplot(gs[0, :])

# print([np.inner(eigvecs[:, 0], eigvecs[:, 0]) for eigvecs in coll_eigvecs])
# print([
#     np.dot(grad, eigvecs[:, 0])
#     for grad, eigvecs in zip(coll_grad, coll_eigvecs)
# ])
idxs = 2*[slice(0, 2)] + (len(coll_eigvecs)-1)*[slice(0, 1)]
zs = [
    grad - np.sum(
        np.inner(grad, eigvecs[:, idx].T)*eigvecs[:, idx], axis=-1
    )
    for grad, eigvecs, idx in zip(coll_grad, coll_eigvecs, idxs)
]
# print([np.linalg.norm(z) for z in zs])

lmbdas = [eigvals[0] for eigvals in coll_eigvals]
artists = plot_triplots(fig, axs, [coords]*len(zs), [cells]*len(zs), zs)
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontour', levels=[0], colors='k')
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontourf')
fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, basis_name, lmbda in zip(axs, basis_names, lmbdas):
    text = f"{basis_name}"
    ax.annotate(text, (0, 1.05), xycoords=ax.transAxes)
    ax.set_aspect(1)

for ax in axs[1:]:
    ax.yaxis.set_tick_params(labelleft=False)

for ax in axs:
    ax.set_xlabel("x [cm]")

axs[0].set_ylabel("y [cm]")

ax_cbar.set_xlabel("${{dp_\\mathrm{on}}} / {{dE}} (I-\sum_i {\Delta E}_i {\Delta E}_i^{T})$ [$\\mathrm{Pa}/\\mathrm{kPa}$]")
# fig.tight_layout()

fig.savefig(f"{FIG_DIR}/GradientWOHessianComparisonAcrossSubspaces.{FIG_EXT}")

In [ ]:
N_MODE = 2
fig = plt.figure(figsize=(FIG_LX_WIDE, 0.4*FIG_LY_MAX), constrained_layout=True)
gs = gridspec.GridSpec(
    1+N_MODE, len(coll_eigvecs), height_ratios=[0.05]+[1]*N_MODE,
    figure=fig
)

axs = np.array([
    [fig.add_subplot(gs[i, n]) for n in range(gs.ncols)]
     for i in range(1, N_MODE+1)
])
ax_cbar = fig.add_subplot(gs[0, :])

for nmode in range(N_MODE):
    zs = [eigvecs[:, nmode] for eigvecs in coll_eigvecs]
    # print(zs[0].shape)
    # print([(z.max(), z.min()) for z in zs])
    zs_norm = [z/(z.max()-z.min()) for z in zs]
    # print([(z.max(), z.min()) for z in zs_norm])
    zs_norm = zs
    lmbdas = [eigvals[nmode] for eigvals in coll_eigvals]
    artists = plot_triplots(
        fig, axs[nmode, :], [coords]*len(zs), [cells]*len(zs), zs_norm
    )

    for ax, lmbda in zip(axs[nmode, :].flat, lmbdas):
        text = f"$\\Lambda = {lmbda:.1f}$"
        ax.annotate(text, (0.05, 0.95), va='top', xycoords=ax.transAxes)

# plot_triplots(fig, axs, [coords]*len(zs), cells, zs, plot_type='tricontourf', levels=[0])
fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, basis_name, lmbda in zip(axs[0, :], basis_names, lmbdas):
    text = f"{basis_name}"
    ax.annotate(text, (0, 1.05), xycoords=ax.transAxes)

for ax in axs.flat:
    ax.set_aspect(1)

for ax in axs[:, 1:].flat:
    ax.yaxis.set_tick_params(labelleft=False)

for ax in axs[:-1, :].flat:
    ax.xaxis.set_tick_params(labelbottom=False)

for ax in axs[-1, :].flat:
    ax.set_xlabel("x [cm]")

for ax in axs[:, 0].flat:
    ax.set_ylabel("y [cm]")

ax_cbar.set_xlabel("Eigenvectors of ${{d^2p_\\mathrm{on}}} / {{dE^2}}$ [$\mathrm{Pa}/\mathrm{kPa}^2$]")

fig.savefig(f"{FIG_DIR}/EigenmodeComparisonAcrossSubspaces.{FIG_EXT}")

##### Compare nonlinear onset pressure variation

In [ ]:
# Directions of principal sensitivities to compare
dirs = [
    eigvecs[:, 0] for eigvecs in coll_eigvecs
]

alphas = np.linspace(-5, 30, 11)
_alphas = np.linspace(-5, 10, 11)

pon_base = xhopf_base.sub['psub'][0]
grads_alpha = [grad_pon.dot(e) for e in dirs]
hesss_alpha = [eigvals[0] for eigvals in coll_eigvals]

In [ ]:
ponsets_vs_dirs = []
for z in tqdm(dirs):
    dparam = transform.x.copy()
    dparam[:] = 0
    dparam['emod'][:] = z
    params = [param_base + alpha*dparam for alpha in alphas]
    prop = [transform.apply(_param) for _param in params]

    def _solve_hopf_by_newton(hopf, xhopf_0, prop, **kwargs):
        xhopf, info = libhopf.solve_hopf_by_newton(
            hopf, xhopf_0, prop, **kwargs
        )
        if info['status'] != 0:
            print("Newton solver for Hopf system did not converge")
        return xhopf

    ponsets = np.array([
        _solve_hopf_by_newton(HOPF_MODEL, xhopf_base, prop).sub['psub'][0]
        for prop in tqdm(prop)
    ])

    ponsets_vs_dirs.append(ponsets)

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(FIG_LX_WIDE, FIG_LY)
)

for color, basis_name, direction, ponsets in zip(COLOR_CYCLE, basis_names, dirs, ponsets_vs_dirs):
    ax.plot(alphas, ponsets/10, label=basis_name, color=color)

for color, basis_name, grad_alpha, hess_alpha in zip(COLOR_CYCLE, basis_names, grads_alpha, hesss_alpha):
    ax.plot(
        _alphas, (pon_base + grad_alpha*_alphas + 1/2*hess_alpha*_alphas**2)/10,
        ls='-.', color=color, alpha=0.5, lw=5
    )

ax.set_ylabel("$p_\\mathrm{{on}}$ [Pa]")
ax.set_xlabel("$\\alpha$ []")

labels = (
    [f"{label}" for label in basis_names]
    + ["Taylor model", "nonlinear"]
)
handles = (
    [mpl.lines.Line2D([0], [0], color=color) for color in COLOR_CYCLE[:len(basis_names)]]
    + [mpl.lines.Line2D([0], [0], color='k', ls=ls, lw=lw) for ls, lw in zip(['-', '-.'], [1.5, 5])]
)
ax.legend(handles, labels)

fig.savefig(f'{FIG_DIR}/OnsetPressureVariationComparisonBetweenSubspacesAlong2ndOrderSensitivity.{FIG_EXT}')

#### Structural eigenmodes along principal sensitivity

In [ ]:
# Generate a list of properties along a perturbation direction
alphas = np.linspace(-10, 10, 6)
dparam = transform.x.copy()
dparam[:] = 0
dparam['emod'] = Z_EIG[:, 0]

params = [param_base + alpha*dparam for alpha in alphas]
props = [transform.apply(_param) for _param in params]

In [ ]:
[prop['emod'][:].max() for prop in props]

In [ ]:
def modal_decomp(model, prop):
    """
    Return the modal decomposition of the model
    """
    state_fp_ini = model.state.copy()
    state_fp_ini[:] = 0
    control = model.control
    control[:] = 0
    control['psub'] = 1e-5
    state_fp, info = libhopf.solve_fp(model, control, prop, state_fp_ini)
    # print(info)

    evals, evecs_r, evecs_i = libhopf.solve_linear_stability(model, state_fp, control, prop)
    return evals, evecs_r, evecs_i

In [ ]:
evals_trend = []
nmode = 5
for prop in props:
    evals, evecs_r, evecs_i = modal_decomp(HOPF_MODEL.res, prop)
    # print(evals)
    evals_trend.append(evals[:nmode*2:2])

# The shape is (# parameter points, # modes)
evals_trend = np.array(evals_trend)

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(FIG_LX, FIG_LY)
)

for n in range(nmode):
    ax.plot(
        alphas, evals_trend[:, n].imag/2/np.pi,
        label=f"Mode {n:d}"
    )

ax.legend()
ax.set_ylabel("Frequency [Hz]")
ax.set_xlabel("$\\alpha$")

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/StructuralModesAlong2ndOrderSensitivity.{FIG_EXT}')

In [ ]:
# Plot modes at the base point
AMP = 500
N_PHASE = 5
N_MODE = 4
PHASES = np.linspace(0, 1, N_PHASE+1)[:N_PHASE]

VERTS_MED = SDOF_TO_VERT[HOPF_MODEL.res.fsimap.dofs_solid]
# VERTS_MED =
# TSHAPE_PARAM =

fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY))
gs = gridspec.GridSpec(N_MODE, N_PHASE, figure=fig)
axs = np.array(
    [[fig.add_subplot(gs[i, j]) for j in range(gs.ncols)] for i in range(gs.nrows)]
)

evals, evecs_r, evecs_i = modal_decomp(HOPF_MODEL.res, prop_base)
for nmode in range(N_MODE):
    print(evals[2*nmode])
    er = evecs_r[2*nmode]['u'][:]
    ei = evecs_i[2*nmode]['u'][:]

    for ax, phase in zip(axs[nmode, :], PHASES):
        # print(ax, phase)
        dcoords = AMP*np.real(
            (er + 1j*ei)*np.exp(phase*2*np.pi*1j)
        )
        dcoords = dcoords[VERT_TO_VDOF].reshape(-1, 2)
        coords_disp = coords+dcoords
        ax.triplot(*coords_disp.T, cells, lw=0.5)

        ax.plot(*coords[VERTS_MED].T, color='k', lw=0.5)

for ax in axs.flat:
    ax.set_adjustable('box')
    ax.set_aspect(1)
    ax.set_xlim(-0.2, 0.9)
    ax.set_ylim(0, 0.6)

for ax in axs[:, 1:].flat:
    ax.yaxis.set_tick_params(labelleft=False)

for ax in axs[:-1, :].flat:
    ax.xaxis.set_tick_params(labelbottom=False)

fig.savefig(f'{FIG_DIR}/StructuralModes.{FIG_EXT}')

#### Compare nonlinear onset pressure variation and taylor sensitivity model

In [ ]:
ZEIG = form_basis(eigvecs)

# Find eigendecomposition in a given basis
Z = form_basis(eigvecs[:2])
# Z = B_BC_CINFSUP_BINFSUP
r_eigvals, r_eigvecs = np.linalg.eigh((Z.T@ZEIG) @ np.diag(eigvals) @ (ZEIG.T@Z))
idx_sort = argsort(r_eigvals, sort_by='real')[::-1]
r_eigvals, r_eigvecs = r_eigvals[idx_sort], r_eigvecs[idx_sort]

print(r_eigvals)
print(r_eigvecs)

In [ ]:
## Sanity check that minimum corresponds to large deviation
dr = np.linalg.solve(np.diag(r_eigvals), -r_eigvecs.T @ (grad_pon @ Z))

da = r_eigvecs@dr
print(da)

In [ ]:
# np.linalg.norm(r_eigvecs[:, 1])

In [ ]:
xs = np.linspace(-5, 5, 6)
# xs = np.linspace(-1, 1, 11)

dirs = []

nmode = 1
for n in tqdm(range(nmode)):
    dparam = transform.x.copy()
    dparam[:] = 0
    dparam['emod'] = Z@r_eigvecs[:, n]

    def _solve(model, xhopf, param, dparam):
        dprop = transform.apply_jvp(param, dparam)

        xhopf_n, info = libhopf.solve_hopf_by_newton(
            HOPF_MODEL, xhopf, prop_base+dprop
        )
        if info['status'] != 0:
            print("didn't converge!")
        return xhopf_n

    ponsets = [
        _solve(HOPF_MODEL, xhopf_base, param_base, x*dparam).sub['psub'][0]
        for x in tqdm(xs)
    ]

    dirs.append(np.array(ponsets))

In [ ]:
fig, ax = plt.subplots(1, 1)

colors = mpl.rcParams['axes.prop_cycle'].by_key()['color']
for n, (eigval, r_eigvec, ponsets) in enumerate(zip(r_eigvals, r_eigvecs.T, dirs)):
    color = colors[n]

    con = xhopf_base.sub['psub'][0]
    lin = (grad_pon).dot(Z@r_eigvec)
    quad = eigval
    print(eigval)

    ax.plot(xs, np.array(ponsets)/10, color=color, label='Exact')
    ax.plot(
        xs, (con + lin*xs + 1/2*quad*xs**2)/10,
        color=color, ls='-.', label='Taylor approximation'
    )

ax.set_xlabel("$a$")
ax.set_ylabel("$p_\mathrm{onset}$ [Pa]")
ax.legend()

### Effect of moving separation point

In [ ]:
emod_cov, emod_bod = (2*1e4, 6*1e4)
clscale = 0.5
output_dir = 'out/'

LAYER_TYPE = 'discrete'
# NUM_SEP = 2

# Choose between plotting the old single split geometry or new 6-way split geometry
# mesh_name = f'M5_CB_GA3_CL{clscale:.2f}_split'
# sep_points = ['separation-inf', 'separation-mid']
mesh_name = f'M5_CB_GA3_CL{clscale:.2f}_split6'
sep_points = [f'sep{n}' for n in (1, 2)]
NUM_SEP = len(sep_points)
params = [
    DEFAULT_PARAM.substitute({
        'MeshName': mesh_name,
        'LayerType': LAYER_TYPE,
        'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure',
        'EigTarget': 'LARGEST_MAGNITUDE', 'SepPoint': sep_point
    })
    for sep_point in sep_points
]
models = [
    HOPF_MODELS[param['MeshName']] for param in params
]
sep_values = [
    model.res.solid.residual.mesh_function_label_to_value('vertex')[param['SepPoint']]
    for model, param in zip(models, params)
]
sep_verts = [
    model.res.solid.residual.mesh_function('vertex').where_equal(value)[0]
    for model, value in zip(models, sep_values)
]
sep_coords = [
    model.res.solid.residual.mesh().coordinates()[idx]
    for idx, model in zip(sep_verts, models)
]

Ms = [MS_DG0[param['MeshName']] for param in params]
cset_coords = [model.res.solid.residual.mesh().coordinates() for model in models]
cset_cells = [model.res.solid.residual.mesh().cells() for model in models]

print(models[0].res.solid.residual.mesh_function_label_to_value('cell'))

NUM_EIG = 2

coll_eigvecs = [
    form_basis([
        evec['emod'][:] for evec in
        load_block_array(
            param, 'hess_param', load_dir=output_dir
        )[:]
    ])
    for param in params
]

coll_eigvals = [
    load_scalar_array(
        param, 'eigvals', load_dir=output_dir
    )[:]
    for param in params
]

if emod_cov == 2e4:
    idxs_sort = [np.argsort(np.abs(eigvals))[::-1] for eigvals in coll_eigvals]
else:
    idxs_sort = [np.argsort(eigvals)[::-1] for eigvals in coll_eigvals]
coll_eigvals = [eigvals[idx_sort] for idx_sort, eigvals in zip(idxs_sort, coll_eigvals)]
coll_eigvecs = [eigvecs[:, idx_sort] for idx_sort, eigvecs in zip(idxs_sort, coll_eigvecs)]

# Flip the sign of eigenvectors to make sure they're consistent with those in the 3D quadratic model figure
B_BCs = [
    make_structured_basis_matrix(
        model.res.solid.residual.form['coeff.prop.emod'].function_space(),
        model.res.solid.residual.mesh(),
        model.res.solid.residual.mesh_function('cell'),
        model.res.solid.residual.mesh_function_label_to_value('cell'), 'BC'
    )
    for model in models
]
signs = [np.sign(np.dot(B_BC[:, 1], A[:, :5])) for A, B_BC in zip(coll_eigvecs, B_BCs)]
coll_eigvecs = [sign*A[:, :5] for sign, A in zip(signs, coll_eigvecs)]

In [ ]:
fig = plt.figure(figsize=(FIG_LX, 0.8*FIG_LY), constrained_layout=True)
gs = gridspec.GridSpec(
    1+NUM_EIG, NUM_SEP,
    figure=fig, height_ratios=[0.05]+[1]*NUM_EIG
)

axs = np.array(
    [[fig.add_subplot(gs[i, j]) for j in range(gs.ncols)] for i in range(1, gs.nrows)],
    dtype=object
)
ax_cbar = fig.add_subplot(gs[0, :])

for ax in axs.flat:
    ax.set_aspect(1)
    # ax.set_ylim(0.35, 0.55)

# IDX = 0
zs = [
    evecs[:, idx_eig] for idx_eig, evecs
    in itertools.product(range(NUM_EIG), coll_eigvecs)
]
scales = [z.max()-z.min() for z in zs]
# scales = [1]*len(zs)
zs_norm = [z/scale for z, scale in zip(zs, scales)]

artists = plot_triplots(
    fig, axs.flat, cset_coords*NUM_EIG, cset_cells*NUM_EIG, zs_norm
)

for axs_col, eigvals in zip(axs.T, coll_eigvals):
    for ax, eigval in zip(axs_col, eigvals):
        ax.annotate(f"$\\Lambda = {eigval:.2f}$", (0.05, 1.05), xycoords=ax.transAxes, va='bottom')

for axs_col, sep_coord in zip(axs.T, sep_coords):
    for ax in axs_col:
        for xy in sep_coords:
            ax.plot(*xy, marker='o', ms=3, mfc='k', mec='none')

        ax.plot(*sep_coord, marker='o', ms=10, mfc='none', mec='k')

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

format_axes_grid(axs, "x [cm]", "y [cm]")

## Add inset axes
from mpl_toolkits.axes_grid1.inset_locator import (
    inset_axes, InsetPosition, mark_inset, zoomed_inset_axes
)

# axs_inset = np.array([
#     zoomed_inset_axes(ax, zoom=2, loc='lower left') for ax in axs[0, :]
# ])

axs_inset = np.array([
    zoomed_inset_axes(ax, zoom=2, loc='lower left') for ax in axs.flat
]).reshape(axs.shape)
plot_triplots(
    fig, axs_inset.flat, cset_coords*NUM_EIG, cset_cells*NUM_EIG, zs_norm
)
# plot_triplots(
#     fig, axs_inset[1, :], cset_coords*NUM_EIG, cset_cells*NUM_EIG, zs_norm
# )
for ax_inset, ax_parent in zip(axs_inset.flat, axs.flat):
    ax_inset.set_xlim(0.6, 0.8)
    ax_inset.set_ylim(0.4, ax_parent.get_ylim()[1])
    ax_inset.set_aspect(1)
    ax_inset.yaxis.set_tick_params(labelleft=False)
    ax_inset.xaxis.set_tick_params(labelbottom=False)

    mark_inset(ax_parent, ax_inset, loc1=2, loc2=4, fc='none', ec='k')

for ax_inset_row in axs_inset:
    for ax, sep_coord in zip(ax_inset_row, sep_coords):
        # print(sep_coords)
        for xy in sep_coords:
            ax.plot(*xy, marker='o', ms=3, mfc='k', mec='none')

        ax.plot(*sep_coord, marker='o', ms=10, mfc='none', mec='k')

symb = r"${d^2 p_\mathrm{on}} / {d E^2}$"
unit = r"$\mathrm{Pa}/\mathrm{kPa}^2$"
ax_cbar.set_xlabel(f"Eigenvectors of {symb} [{unit}]")
format_xaxis_top(ax_cbar.xaxis)

fig.savefig(f'{FIG_DIR}/SeparationEffect_LayerType{LAYER_TYPE}_EC{emod_cov:.2e}_EB{emod_bod:.2e}_CL{clscale}.{FIG_EXT}')

### Mesh and jacobian step size dependence

In [ ]:
emod_cov, emod_bod = (6e4, 6e4)
hs = np.array([1e-2, 1e-3, 1e-4, 1e-5])
hs = np.array([1e-2, 1e-3, 1e-4])
# hs = np.array([1e-3, 1e-4])
hs = np.array([1e-2, 1e-3, 1e-4])
clscales = np.array([0.5, 0.25, 0.125])
clscales = np.array([1.0, 0.5, 0.25])
# clscales = np.array([0.5, 0.25])
output_dir = 'out/independence'

# eig_target = 'LARGEST_REAL'
eig_target = 'LARGEST_MAGNITUDE'

NUM_EIG = 5
params = [
    DEFAULT_PARAM.substitute({
        'MeshName': f'M5_CB_GA3_CL{clscale:.2f}', 'H': h,
        'Ecov': emod_cov, 'Ebod': emod_bod,
        'Functional': 'OnsetPressure',
        'EigTarget': eig_target
    })
    for h, clscale in itertools.product(hs, clscales)
]
models = [
    HOPF_MODELS[param['MeshName']] for param in params
]
Ms = [MS_DG0[param['MeshName']] for param in params]
cset_coords = [
    model.res.solid.residual.mesh().coordinates()
    for model in models
]
cset_cells = [
    model.res.solid.residual.mesh().cells()
    for model in models
]

coll_eigvecs = np.array([
    form_basis([
        evec['emod'][:] for evec in
        load_block_array(
            param, 'hess_param', load_dir=output_dir
        )[:NUM_EIG]
    ])
    for param in params
], dtype=object).reshape(len(hs), len(clscales))

coll_eigvals = np.array([
    load_scalar_array(
        param, 'eigvals', load_dir=output_dir
    )[:NUM_EIG]
    for param in params
]).reshape(len(hs), len(clscales), NUM_EIG)

coll_eigvecs.shape

In [ ]:
IDX_EIG = 0

In [ ]:
### Plot eigenvectors with varying step size and mesh size
fig = plt.figure(figsize=(FIG_LX, 0.7*FIG_LY), constrained_layout=True)
gs = gridspec.GridSpec(
    len(hs)+1, len(clscales),
    figure=fig, height_ratios=[0.05]+len(hs)*[1]
)

axs = np.array(
    [[fig.add_subplot(gs[i, j]) for j in range(gs.ncols)] for i in range(1, gs.nrows)],
    dtype=object
)
for ax in axs.flat:
    ax.set_aspect(1)
ax_cbar = fig.add_subplot(gs[0, :])

zs = [A[:, IDX_EIG] for A in coll_eigvecs.flat]
scales = [z.dot(M[:, :].dot(z))**0.5 for M, z in zip(Ms, zs)]
# scales = [abs(z.max()-z.min()) for z in zs]
zs_norm = [z/scale for z, scale in zip(zs, scales)]

lmbdas = [eigval for eigval in coll_eigvals[..., IDX_EIG].flat]
lmbdas_norm = [lmbda/scale**2 for scale, lmbda in zip(scales, lmbdas)]

# Find/plot the location of the maximum for debugging
cell_max_idxs = [np.argmax(np.abs(z)) for z in zs]
cell_max_coords = [
    np.sum(coords[cells[idx]], axis=0)/3
    for coords, cells, idx in zip(cset_coords, cset_cells, cell_max_idxs)
]
ncells = [cells.shape[0] for cells in cset_cells]
# for ax, coords_max in zip(axs.flat, cell_max_coords):
#     ax.plot(*coords_max, marker='o', markersize=10)

artists = plot_triplots(
    fig, axs.flat, cset_coords, cset_cells, zs_norm
    # vmin=-0.001, vmax=0.001
)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
for ax, lmbda in zip(axs.flat, lmbdas_norm):
    ax.annotate(f"$\\Lambda={lmbda:.1f}$", (0.05, 0.95), va='top', xycoords=ax.transAxes)

for ax in axs[:, 0]:
    ax.set_ylabel("y [cm]")

for ax in axs[-1, :]:
    ax.set_xlabel("x [cm]")

for ax in axs[:-1, :].flat:
    ax.xaxis.set_tick_params(labelbottom=False)
for ax in axs[:, 1:].flat:
    ax.yaxis.set_tick_params(labelleft=False)

for ax, h in zip(axs[:, -1], hs):
    ax.annotate(
        f"h: {h:.3e}", (1.05, 0.5),
        rotation=-90, xycoords=ax.transAxes, va='center'
    )

for ax, ncell in zip(axs[0, :], ncells):
    ax.annotate(
        f"$n_\\mathrm{{cell}}$: {ncell:d}", (0.5, 1.05),
        rotation=0, xycoords=ax.transAxes, ha='center'
    )

symb = r"${d^2 p_\mathrm{on}} / {d E^2}$"
unit = r"$\mathrm{Pa}/\mathrm{kPa}^2$"
ax_cbar.set_xlabel(f"Eigenvectors of {symb} [{unit}]")
format_xaxis_top(ax_cbar.xaxis)

fig.savefig(f'{FIG_DIR}/EigVecIndependence_Ecov{emod_cov:.2e}_Ebod{emod_bod:.2e}_{IDX_EIG:d}.{FIG_EXT}')

In [ ]:
fig, ax = plt.subplots(1, 1)

for h, eigvals_trend, eigvecs_trend in zip(hs, coll_eigvals, coll_eigvecs):
    es = [A[:, IDX_EIG] for A in eigvecs_trend]
    # es_scale = [np.abs(e).max() for e in es]
    es_scale = [e.dot(M[:, :].dot(e))**0.5 for M, e in zip(Ms, es)]
    es_normalized = [e/scale for e, scale in zip(es, es_scale)]
    # es_normalized = [e/(e.max()-e.min()) for e in es]
    scales = np.array([np.linalg.norm(e) for e in es_normalized])

    eigvals = eigvals_trend[:, IDX_EIG]

    print(eigvals, scales)
    print(eigvals*scales**2)

    ax.plot(
        1/clscales, eigvals*scales**2
    )

ax.set_xlabel("Mesh size refinement")
ax.set_ylabel(f"Eigenvalue {IDX_EIG+1:d}")

fig.tight_layout()
fig.savefig(f'fig/manuscript/EigValIndependence_Ecov{emod_cov:.2e}_Ebod{emod_bod:.2e}_{IDX_EIG:d}.{FIG_EXT}')

### Sensitivity results for varying linearization points

In [ ]:
# Load results over varying cover/body stiffness

param_option = 'Stiffness'

coll_eigvals = []
coll_eigvecs = []
emods_cov_factor = np.array([1, 1/2, 1/3, 1/4])
emods_bod = np.arange(2, 18, 4) * 10 * 1e3
for emod_cov_factor, emod_bod in itertools.product(emods_cov_factor, emods_bod):
    emod_cov = emod_cov_factor*emod_bod
    eigvals, eigvecs = load_eigenpair(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'ParamOption': param_option}
    )

    coll_eigvals.append(eigvals)
    coll_eigvecs.append(eigvecs)


#### Traction-based shape mode

In [ ]:
sl_residual = HOPF_MODEL.res.solid.residual
func_space = sl_residual.form['coeff.state.u1'].function_space()
coords = func_space.tabulate_dof_coordinates()

mesh = sl_residual.mesh()
mesh_coords = mesh.coordinates()
tri = mpl.tri.Triangulation(
    mesh_coords[:, 0], mesh_coords[:, 1], mesh.cells()
)

fig, axs = plt.subplots(len(emods_cov_factor), len(emods_bod), figsize=(10, 10))

for ax, eigvals, eigvecs in zip(axs.flat, coll_eigvals, coll_eigvecs):
    # print(eigvecs)
    # Plot the original mesh
    ax.triplot(*(mesh_coords).T, mesh.cells(), alpha=0.5, color='k', lw=0.3)

    # Compute/plot the mesh motion corresponding to the traction
    umesh = mesh_traction_to_displacement(transform, eigvecs[0]['tmesh'])
    umesh = umesh[VERT_TO_VDOF].reshape(-1, 2)
    ax.triplot(*(mesh_coords+1*umesh).T, mesh.cells(), lw=0.5)

format_axes_grid(axs, "x [cm]", "y [cm]")
for ax in axs.flat:
    ax.set_aspect(1)

# for ax in axs[:, 0]:
ylabel_text = axs[0, 0].get_ylabel()
# print(type(ylabel_text))
    # ax.text()


#### Linear stability modes vs linearization point

In [ ]:
def _control(xhopf):
    HOPF_MODEL.set_state(xhopf)
    return HOPF_MODEL.res.control.copy()

states_fp = [xhopf[HOPF_MODEL.labels_fp] for xhopf in cset_xhopf]
controls_fp = [_control(xhopf) for xhopf in cset_xhopf]
eigmodes = [
    libhopf.solve_linear_stability(HOPF_MODEL.res, state_fp, control, prop)[0]
    for state_fp, control, prop in tqdm(zip(states_fp, controls_fp, cset_props))
]

In [ ]:
ponsets = np.array([x.sub['psub'][0] for x in cset_xhopf])
fonsets = np.array([x.sub['omega'][0]/2/np.pi for x in cset_xhopf])

In [ ]:
NUM_MODE = 5
eigmodes_ = np.array([x[:2*NUM_MODE:2] for x in eigmodes])

In [ ]:
fig = plt.figure(figsize=(FIG_LX, FIG_LY))
gs = gridspec.GridSpec(2, 1, figure=fig)
axs = np.array([fig.add_subplot(gs[i, 0]) for i in range(gs.nrows)])

axs[0].plot(
    np.arange(eigmodes_.shape[0])+1, ponsets/10,
    label="$p_\\mathrm{on}$"
)
axs[0].set_ylabel("$p_\\mathrm{on}$ [Pa]", color='b')
ax_twin = axs[0].twinx()
ax_twin.plot(
    np.arange(eigmodes_.shape[0])+1, fonsets,
    label="$f_\\mathrm{on}$", color='r'
)
ax_twin.set_ylabel("$f_\\mathrm{on}$ [Hz]", color='r')

for nmode in range(eigmodes_.shape[1]):
    axs[1].scatter(
        np.arange(eigmodes_.shape[0])+1, eigmodes_[:, nmode].imag/2/np.pi,
        label=f"Mode {nmode+1:d}"
    )

axs[1].set_xlabel("Linearization point")
axs[1].set_ylabel("Mode frequency [Hz]")
axs[1].legend()

## Miscellaneous experiments

In [ ]:
MESH_NAME = 'M5_CB_GA3_CL0.50'
mesh_path = f'mesh/{MESH_NAME}.msh'

### Plot onset vibration

In [ ]:
hopf_kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}

# tran_kwargs = {
#     'sep_method': 'arearatio'
# }

tran_kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}

tres = libsetup.load_transient_model(mesh_path, **tran_kwargs)
hres, res, dres = libsetup.load_hopf_model(mesh_path, **hopf_kwargs)

In [ ]:
tran_prop = tres.prop.copy()
hopf_prop = hres.prop.copy()

for prop in (tran_prop, hopf_prop):
    EMOD = 5e3*10
    prop['emod'][:] = EMOD
    prop['eta'][:] = 5
    prop['rho'][:] = 1
    prop['rho_air'][:] = 1.2e-3

    YMAX = tres.solid.mesh.coordinates()[:, 1].max()
    YMID = YMAX + 0.05
    prop['ymid'][:] = YMID
    prop['ycontact'][:] = YMID - 0.01
    prop['kcontact'][:] = 1e12

# tran_prop.print_summary()
if 'r_sep' in tran_prop:
    tran_prop['r_sep'] = 1.0

In [ ]:
emods = [5e3*10, 8e3*10]
hopf_props = []
tran_props = []
for emod in emods:
    for prop in (tran_prop, hopf_prop):
        prop['emod'][:] = emod
    hopf_props.append(hopf_prop.copy())
    tran_props.append(tran_prop.copy())

In [ ]:
xhopfs = []
for prop in hopf_props:
    _psubs = np.arange(50, 500, 50)*10
    xhopf_0 = libhopf.gen_xhopf_0(hres.res, prop, hres.E_MODE, _psubs)
    xhopf = hres.state.copy()
    xhopf[:], solve_info = libhopf.solve_hopf_by_newton(hres, xhopf_0, prop)
    xhopfs.append(xhopf)

In [ ]:
for prop in hopf_props:
    prop.print_summary()

#### Plot fixed points over varying $p_\mathrm{sub}$

In [ ]:
prop = hopf_props[0]

In [ ]:
psubs = [100*10, xhopf['psub'][0]]
controls = [hres.res.control.copy() for _ in psubs]
for control, psub in zip(controls, psubs):
    control['psub'][:] = psub
xfps = [libhopf.solve_fp(hres.res, control, prop)[0] for control in controls]

In [ ]:
CELLS = hres.res.solid.forms['mesh.mesh'].cells()
XY = hres.res.solid.forms['mesh.mesh'].coordinates()
hres.res.solid.forms.keys()
VERT_TO_VDOF = dfn.vertex_to_dof_map(hres.res.solid.forms['coeff.state.u1'].function_space())

In [ ]:
tri = mpl.tri.Triangulation(*(XY).T, CELLS)

In [ ]:
INCH = 1/25.4
fig = plt.figure(figsize=(100*INCH, 130*INCH), constrained_layout=True)

gs = mpl.gridspec.GridSpec(1, 1, figure=fig)
ax = fig.add_subplot(gs[0, 0])

tri_lines, *_ = ax.triplot(tri, marker=None, lw=1)

ax.plot([0, XY[:, 0].max()], [YMID, YMID], color='k', ls='-.')
ax.set_xlim(-0.1, 0.9)
ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")
ax.set_aspect(1)

annotation = ax.annotate("", (0.05, 1.05), ha='left', va='bottom', xycoords=ax.transAxes)

for psub, xfp in zip(psubs, xfps):
    xy = XY + xfp['u'][VERT_TO_VDOF].reshape(-1, 2)
    set_triangulation_line(tri_lines, tri, xy)

    annotation.set_text(f"$p_\mathrm{{sub}}={psub/10:.1f}\,\mathrm{{Pa}}$")

    fig.savefig(f'fig/presentation/OnsetWiggle--psub{psub:.2f}--emod{EMOD:.2e}.svg')


#### Animate stable/unstable oscillations when $p_\mathrm{sub} = p_\mathrm{onset} \pm \Delta p_\mathrm{sub}$

In [ ]:
NUM_FRAME = 5001
DEL_PSUB = 50*10

In [ ]:
## Run transient simulations for each hopf point subglottal pressure
print(emods)
for emod, tran_prop, hopf_prop, xhopf in zip(emods, tran_props, hopf_props, xhopfs):
    # This control is at ponset +
    control = tres.control.copy()
    # control = hres.res.control.copy()
    control['psub'][:] = xhopf.sub['psub'][0] + DEL_PSUB
    print(xhopf.sub['psub'][0])
    # control.print_summary()
    # tran_prop.print_summary()

    # Set the initial state to be the fixed point
    state_fp, solve_info = libhopf.solve_fp(hres.res, control, hopf_prop)
    state_0 = tres.state1.copy()
    state_0[:] = 0
    state_0['u'] = state_fp['u']
    # state_0['q'] = state_fp['q']
    # state_0['p'] = state_fp['p']
    # print(state_0)
    state_0.print_summary()

    # tres.set_ini_state(state_0)
    # tres.set_fin_state(state_0)
    # tres.
    # print(tres.assem_res().norm())

    fname = f'fig/presentation/OnsetWiggle--delpsub{DEL_PSUB:.2f}--emod{emod:.2e}.h5'
    if not isfile(fname):
        with sf.StateFile(tres, fname, mode='a') as f:
            period = 1/(xhopf['omega'][0]/(2*np.pi))
            # Simulate at 0.002 s/60 frames
            _times = np.linspace(0, (1/500)/60*NUM_FRAME, NUM_FRAME)
            times = bv.BlockVector((_times,), labels=(('time',),))
            integrate(tres, f, state_0, [control], tran_prop, times, use_tqdm=True)
    else:
        print(f"{fname} already exists")

In [ ]:
CELLS = hres.res.solid.forms['mesh.mesh'].cells()
XY = hres.res.solid.forms['mesh.mesh'].coordinates()
VERT_TO_VDOF = dfn.vertex_to_dof_map(hres.res.solid.forms['coeff.state.u1'].function_space())
tri = mpl.tri.Triangulation(*(XY).T, CELLS)

In [ ]:
# Set the emod case
emod = emods[0]
xhopf = xhopfs[0]
hopf_prop = hopf_props[0]

# Solve the stable fixed-point at ponset - dpsub
control = hres.res.control.copy()
control['psub'] = xhopf.sub['psub'][0] - DEL_PSUB
state_fp, solve_info = libhopf.solve_fp(hres.res, control, hopf_prop)

print(solve_info)

In [ ]:
fig = plt.figure(figsize=(300*INCH, 130*INCH), constrained_layout=True)

gs = mpl.gridspec.GridSpec(1, 2, figure=fig)
axs = [fig.add_subplot(gs[0, ii]) for ii in range(gs.ncols)]

tri_lines = [ax.triplot(tri, marker=None, lw=1)[0] for ax in axs]

# Set the stable fixed point displacements
xy = XY + state_fp['u'][VERT_TO_VDOF].reshape(-1, 2)
set_triangulation_line(tri_lines[0], tri, xy)

for ax in axs:
    ax.plot([0, XY[:, 0].max()], [YMID, YMID], color='k', ls='-.')

for ax in axs:
    ax.set_aspect(1)
    ax.set_xlim(-0.1, 0.9)
    ax.set_xlabel("x [cm]")

axs[0].set_ylabel("y [cm]")
for ax in axs[1:]:
    ax.yaxis.set_tick_params(labelleft=False)

for ax, sign in zip(axs, ('-', '+')):
    annotation = ax.annotate("", (0.05, 1.05), ha='left', va='bottom', xycoords=ax.transAxes)
    annotation.set_text(f"$p_\mathrm{{on}} {sign} 50\,\mathrm{{Pa}}$")

In [ ]:
with sf.StateFile(tres, f'fig/presentation/OnsetWiggle--delpsub{DEL_PSUB:.2f}--emod{emod:.2e}.h5', mode='r') as f:
    f.get_state(0).print_summary()

In [ ]:
SCALE = 10
with sf.StateFile(tres, f'fig/presentation/OnsetWiggle--delpsub{DEL_PSUB:.2f}--emod{emod:.2e}.h5', mode='r') as f:
    u_fp = f.get_state(0).sub['u']

    def animate(frame):
        u = f.get_state(frame).sub['u']
        du = u - u_fp
        # print(np.linalg.norm(du))

        xy = XY + (SCALE*du + u_fp)[VERT_TO_VDOF].reshape(-1, 2)
        # xy = XY + (SCALE*u)[VERT_TO_VDOF].reshape(-1, 2)

        tri_edges = tri.edges
        tri_line_x = np.insert(xy[:, 0][tri_edges], 2, np.nan, axis=1)
        tri_line_y = np.insert(xy[:, 1][tri_edges], 2, np.nan, axis=1)
        tri_lines[-1].set_xdata(tri_line_x.ravel())
        tri_lines[-1].set_ydata(tri_line_y.ravel())
        # BUG: This doesn't work for some reason!?
        # set_triangulation_line(tri_lines[-1], tri, xy)

    animation = mpl.animation.FuncAnimation(fig, animate, frames=tqdm(range(0, 5000, 5)))
    animation.save(f'fig/presentation/OnsetWiggle--delpsub{DEL_PSUB:.2f}--emod{emod:.2e}.mp4', fps=60, dpi=200)

#### Compare onset wiggles between two different stiffnesses

In [ ]:
emods = 10*np.array([5e3, 8e3])

In [ ]:
ponsets = []
for emod in emods:
    prop['emod'][:] = emod
    psubs = np.arange(50, 800, 50)*10
    xhopf_0 = libhopf.gen_xhopf_0(hres.res, prop, hres.E_MODE, psubs)

    xhopf = hres.state.copy()
    xhopf[:], solve_info = libhopf.solve_hopf_by_newton(hres, xhopf_0, prop)
    ponsets.append(xhopf['psub'][0])

In [ ]:
ponsets

In [ ]:
fig = plt.figure(figsize=(300*INCH, 130*INCH), constrained_layout=True)

gs = mpl.gridspec.GridSpec(1, len(emods), figure=fig)
axs = [fig.add_subplot(gs[0, ii]) for ii in range(gs.ncols)]

tri_lines = [ax.triplot(tri, marker=None, lw=1)[0] for ax in axs]

for ax in axs:
    ax.plot([0, XY[:, 0].max()], [YMID, YMID], color='k', ls='-.')

for ax in axs:
    ax.set_aspect(1)
    ax.set_xlim(-0.1, 0.9)
    ax.set_xlabel("x [cm]")

axs[0].set_ylabel("y [cm]")
for ax in axs[1:]:
    ax.yaxis.set_tick_params(labelleft=False)

for ax, emod, psub in zip(axs, emods, ponsets):
    annotation = ax.annotate("", (0.05, 1.05), ha='left', va='bottom', xycoords=ax.transAxes)
    annotation.set_text(f"$(E, p_\mathrm{{sub}})=({emod/10/1e3:.1f}\,\mathrm{{kPa}}, {psub/10:.1f}\,\mathrm{{Pa}})$")

In [ ]:
SCALE = 1
with sf.StateFile(tres, f'fig/presentation/OnsetWiggle--psub{ponsets[0]+3000:.2f}--emod{emods[0]:.2e}.h5', mode='r') as f1, \
    sf.StateFile(tres, f'fig/presentation/OnsetWiggle--psub{ponsets[1]+3000:.2f}--emod{emods[1]:.2e}.h5', mode='r') as f2:
    def animate(frame):
        for tri_line, f in zip(tri_lines, (f1, f2)):
            u = f.get_state(frame).sub['u']

            xy = XY + SCALE*u[VERT_TO_VDOF].reshape(-1, 2)
            tri_edges = tri.edges
            tri_lines_x = np.insert(xy[:, 0][tri_edges], 2, np.nan, axis=1)
            tri_lines_y = np.insert(xy[:, 1][tri_edges], 2, np.nan, axis=1)
            tri_lines[-1].set_xdata(tri_lines_x.ravel())
            tri_lines[-1].set_ydata(tri_lines_y.ravel())

    animation = mpl.animation.FuncAnimation(fig, animate, frames=range(NUM_FRAME))

    animation.save(f'fig/presentation/OnsetWiggleComparison.mp4', fps=60, dpi=300)

### Growth rate plot for generic model

Plots of the growth rate vs subglottal pressure.

In [ ]:
MESH_NAME = 'M5_CB_GA3_CL0.50'
mesh_path = f'mesh/{MESH_NAME}.msh'

kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}
model_tran = load_tran(mesh_path, **kwargs)

# load the Hopf models
res_hopf, res, dres = load_hopf(mesh_path, **kwargs)

In [ ]:
for key, subvec in res.prop.items():
    print(f"({key}) mean, min, max: {np.mean(subvec)}, {np.min(subvec.array)}, {np.max(subvec.array)}")

for key, subvec in res.control.items():
    print(f"({key}) mean, min, max: {np.mean(subvec)}, {np.min(subvec.array)}, {np.max(subvec.array)}")

In [ ]:
psubs = np.arange(0, 800, 20)*10
# psubs = np.arange(700, 800, 2)*10
modes = [libhopf.solve_least_stable_mode(res, psub)[0] for psub in tqdm(psubs)]
modes = np.array(modes)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5, 5), sharex=True)

axs[0].plot(psubs, modes.real)
axs[1].plot(psubs, modes.imag)

axs[0].axhline(0, color='k', ls='-.')

axs[0].set_ylabel("$\omega_{real}$")
axs[1].set_ylabel("$\omega_{imag}$")
axs[1].set_xlabel("$p_{sub}$")

# axs[0].legend()
# axs[1].legend()

fig.tight_layout()

### Bifurcation plots for smooth minimum separation fluid models

In [ ]:
# Set the model properties and sanity check the values
_region_to_dofs = meshutils.process_celllabel_to_dofs_from_forms(
    res.solid.forms, res.solid.forms['fspace.scalar']
)
prop = libsetup.set_prop(res.prop, _region_to_dofs, res)

prop['zeta_min'] = 1e-8
prop['zeta_sep'] = 1e-4
res.set_prop(prop)

# proplabel_to_norm = {label: subvec.norm() for label, subvec in prop.items()}
# pprint(proplabel_to_norm)

proplabel_to_max = {label: subvec.max() for label, subvec in prop.items()}
pprint(proplabel_to_max)
print(res.solid.forms['mesh.mesh'].coordinates()[..., 1].max())

#### Plot the growth rate vs $p_{sub}$

In [ ]:
zeta_min = 1e-6
zeta_sep = 1e-2
prop = res.prop
prop['zeta_min'] = zeta_min
prop['zeta_sep'] = zeta_sep
res.set_prop(prop)

# psubs = np.arange(585, 590, 0.1)*10
psubs = np.arange(700, 800, 10)*10
# psubs = np.arange(700, 800, 2)*10
modes = [libhopf.solve_least_stable_mode(res, psub)[0] for psub in tqdm(psubs)]
modes = np.array(modes)

In [ ]:
def jac_block_norms(res, psub):
    xfp, info = libhopf.solve_fp(res, psub)
    res.set_state(xfp)
    res.control['psub'].array[0] = psub
    res.set_control(res.control)

    dres_dstate = res.assem_dres_dstate()
    return tuple([submat.norm() for submat in dres_dstate.subarrays_flat])

block_norms = [jac_block_norms(res, psub) for psub in tqdm(psubs)]
block_labels = [','.join(multi_label) for multi_label in itertools.product(res.state.labels[0], res.state.labels[0])]

block_norms = np.array(block_norms)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(5, 5), sharex=True)

ii_max = np.argmax(modes.real)
print(psubs[ii_max])

axs[0].plot(psubs, modes.real)
axs[0].axhline(0, color='k', ls='-.')
axs[0].set_ylabel("$\omega_{real}$")

axs[1].plot(psubs, modes.imag)
axs[1].set_ylabel("$\omega_{imag}$")

for norms, label in zip(block_norms.T, block_labels):
    axs[2].plot(psubs, norms, label=label)
axs[2].set_ylabel("$|| \\frac{dF}{dx} ||$")
axs[2].set_xlabel("$p_{sub}$")
# axs[2].set_yscale('log')
axs[2].legend()

# axs[0].set_ylim(-10, 50)

fig.savefig(f'fig/GrowthRatevsPsub_zetamin{zeta_min:.2e}_zetasep{zeta_sep:.2e}_closeup.png', dpi=200)

### Strange bifurcation behaviour for smoothed minimum separation models

Rapid changes in eigenvalues can occur with small changes in $p_{sub}$ when using a smooth minimum approximation. This happens when the smooth minimum approximation approaches the true minimum where small changes in areas rapidly shift weights between nodes.

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True)

s = np.linspace(0, 1, 12)
a = (s-0.5)**2 + 0.1
da = 1e-2*s

def smoothmin(a, zeta=1e-4):
    log_w = -a/zeta
    log_w = log_w - log_w.max()
    print(log_w)
    _w = np.exp(log_w)
    print(_w.max())
    return _w/np.sum(_w)

axs[0, 0].plot(s, a, marker='.')
axs[0, 0].set_title("a [cm]")
axs[1, 0].plot(s, smoothmin(a))
axs[1, 0].set_ylim(0, 1)

axs[0, 0].set_ylabel("Area [cm]")
axs[1, 0].set_ylabel("smoothmin weight")

axs[0, 1].plot(s, a+da, marker='.')
axs[0, 1].set_title("a + da [cm]")
axs[1, 1].plot(s, smoothmin(a+da))
axs[1, 1].set_ylim(0, 1)

# axs[0, 0].yaxis.
for ax_row in axs:
    for col in range(1, ax_row.size):
        ax_row[0].sharey(ax_row[col])
        ax_row[col].yaxis.set_tick_params(labelleft=False)